In [ ]:
from openai import OpenAI
import os
import urllib.request
import subprocess
import time
import keyboard
from abc import ABC, abstractmethod
import json
import http.server
from http.server import HTTPServer
import threading
import socket
import platform
import win32com.client
import psutil
import re
import paramiko
import queue
import math
# CONFIGURATION:

# Set to False after the first run to avoid repeating the setup (Set to False if you changed any VM or Docker relevant configuration)
FIRST_TIME_SETUP = False

# General
MAX_NUMBER_OF_PROBLEMS_TO_TEST_IN_BATCH = 48 # To separate the testing into batches to avoid long running times
EXPERIMENT_NAME = 'Experiment 1' # If there are multiple experiments, their results will be stored in the same table, so this is used to differentiate them
START_FROM_PROBLEM_NAME = 'FindIPAddress' # Set this if the script didn't finish or batches are being used and you want to continue from a specific problem without losing the collected results from previous problems
ATTEMPTS = 2 # Number of attempts to generate a solution for a problem
REDO_PROBLEMS = False # If set to True, the problems that were already tested (partially or fully) will be retested

# VM setup:
VM_NAME = 'debian_vbox'
DEBIAN_ISO = 'https://cdimage.debian.org/debian-cd/current/amd64/iso-cd/debian-12.9.0-amd64-netinst.iso' #'https://cdimage.debian.org/mirror/cdimage/archive/12.1.0/i386/iso-cd/debian-12.1.0-i386-netinst.iso'
ISO_PATH = 'iso'
VM_FOLDER = 'debian'
SHARED_FOLDER = 'scripts'
PRESEED_FOLDER = 'preseed'
PRESEED_PORT = 8080 # If this port is being used already, it can be changed
# IMPORTANT: Change the guest additions url to the version of VirtualBox you are using
GUEST_ADDITIONS_ISO_URL = 'https://download.virtualbox.org/virtualbox/7.1.6/VBoxGuestAdditions_7.1.6.iso'

# Script generation:
HYPERPARAMERTRIZATIONS = [
    {'temperature': 0, 'top_p': 0.1},
    {'temperature': 0.2, 'top_p': 0.4},
    {'temperature': 0.7, 'top_p': 0.9},
]
NOT_A_BASH = 'NOT_A_BASH_SCRIPT'
SYSTEM_CONTEXT = """You answer requests with only a block of Bash code for Debian
Linux to achieve the requested effect. If prompted again with issues about the
code, you provide an improved code. You assume that you are working as
root. You may use any utilities, that can be installed available via `apt install`, assume that they are already installed."""
KEYS_DIR = 'apiKeys'
# List of LLMs to test (OpenAI models don't require a url, rest are specified)
MODELS = [
    {'name': 'gpt-4o', 'keyFile': 'openai.key', 'url': None},
    {'name': 'o1-mini', 'keyFile': 'openai.key', 'url': None},
    {'name': 'qwen/qwen2.5-coder-32b-instruct', 'techninal_name':'Qwen/Qwen2.5-Coder-32B-Instruct', 'keyFile': 'deepinfra.key', 'url':'https://api.deepinfra.com/v1/openai'},
    {'name': 'llama3.3-70b', 'keyFile': 'llama.key', 'url':'https://api.llama-api.com'},
    {
        'name': 'deepseek-r1',
        'techninal_name': 'deepseek-reasoner',
        'keyFile': 'deepseek.key',
        'url': 'https://api.deepseek.com'
    },
    {    
        'name': 'deepseek-v3',
        'techninal_name': 'deepseek-chat',
        'keyFile': 'deepseek.key',
        'url': 'https://api.deepseek.com'
    },
    {'name': 'gpt-3.5-turbo', 'keyFile': 'openai.key', 'url': None},
]

MODEL_MIRRORS = [
    {'name': 'deepseek-r1', 'keyFile': 'llama.key', 'url': 'https://api.llama-api.com'},
    {'name': 'deepseek-v3', 'keyFile': 'llama.key', 'url': 'https://api.llama-api.com'},
]

for model in MODELS:
    with open(f'{KEYS_DIR}/{model["keyFile"]}', 'r') as f:
        model['key'] = f.read().strip()

for model in MODEL_MIRRORS:
    with open(f'{KEYS_DIR}/{model["keyFile"]}', 'r') as f:
        model['key'] = f.read().strip() 

# Testing:
HOST_SCRIPTS_DIR = 'scripts'
SHELLCHECK_VM_SCRIPTS_DIR = '/usr/scripts' # Script path in the shellcheck Docker container
DEBIAN_VM_SCRIPTS_DIR = f'/media/sf_{SHARED_FOLDER}' # Shared folder in the VM
START_CHECKPOINT = 'start_checkpoint' # Debian VM checkpoint used in the beggining of the setup for each test case, also to prevent having to repeat Debian installation
EXPERIMENT_PROJECT_REPO='https://github.com/JanisAndisLapans/BashExperiment'
# Utilities that will be needed to setup the tests (all are installed before START_CHECKPOINT to speed up tests)
STARTING_UTILITIES = ['git', 'openssh-server', 'apt-file', 'jq', 'trivy', 'shellcheck', 'p7zip-full', 'wodim', 'csvtool', 'libncursesw5-dev', 'autotools-dev', 'autoconf', 'automake', 'build-essential', 'net-tools', 'ca-certificates', 'curl', 'gnupg', 'nginx', 'gpg']

SKIP_CORRECT_SCRIPTS = False # If set to True, the prewritten correct scripts will not be tested, this can save time if they were already executed before but is not recommended otherwise since they insure the tests are not faulty

ISSUE_TYPE_WEIGHTS = {
    'CONDITION_NOT_MET': 1,
    'CONDITION_PARTIALLY_MET': 0.5,
    'TOO_MUCH_DONE': 1.5,
    'MINOR_SIDE_EFFECT': 0.25,
}


# BASIC FUNCTIONS:
def run_command(command):
    res = subprocess.run(command,
                         shell=True, 
                         stderr=subprocess.PIPE, 
                         stdout=subprocess.PIPE)
    if res.returncode != 0 and ('docker' not in command or res.returncode != 1):
        raise Exception(f'Error running command (return code {res.returncode}): {res.stderr}')
    
    return type('', (object,),{
        "stdout": res.stdout.decode() if res.stdout != None else None,
        "stderr": res.stderr.decode() if res.stderr != None else None
    })

cached_ip = None
def create_ssh_connection(user, password):
    # Initialize SSH client
    client = paramiko.SSHClient()
    client.set_missing_host_key_policy(paramiko.AutoAddPolicy())

    # Connect to the VM
    grep_utility = 'findstr' if platform.system() == 'Windows' else 'grep'
    attempts = 2
    ip = None
    exception = None
    while ip == None and attempts > 0:
        try:
            if cached_ip != None:
                ip = cached_ip
            else:
                ip = re.search("'([0-9\.]+)'", run_command(f'VBoxManage guestproperty enumerate "debian_vbox" | {grep_utility} IP').stdout).group(1)
            client.connect(ip, username=user, password=password, timeout=10)
        except Exception as e:
            ip = None
            cached_ip = None
            attempts -= 1
            time.sleep(5)
            exception = e
    if ip == None:
        raise exception
    
    cached_ip = ip

    return client

def debian_exec (*commands, client=None, raise_errors=True):
    if client == None:
        client = create_ssh_connection('root', 'preseed')
    
    _, stdout, stderr = client.exec_command(' && '.join(commands))
    exit_code = stdout.channel.recv_exit_status()
    if raise_errors and exit_code != 0:
        client.close()
        raise Exception(f'Error running command {commands}: (exit code: {exit_code}) {stderr.read().decode()}')
    
    ret = type('', (object,),{
        "stdout": stdout.read().decode() if stdout != None else None,
        "stderr": stderr.read().decode() if stderr != None else None
    })

    client.close()

    return ret

def save_script (script_name, script_content):
    with open(f'{HOST_SCRIPTS_DIR}/{script_name.replace("/", "_")}.sh', 'w', newline='\n') as f:
        f.write(script_content.replace('\r\n', '\n'))

def read_script (script_name):
    if not os.path.exists(f'{HOST_SCRIPTS_DIR}/{script_name.replace("/", "_")}.sh'):
        return None
    with open(f'{HOST_SCRIPTS_DIR}/{script_name.replace("/", "_")}.sh', 'r', newline='\n') as f:
        return f.read()
    
def append_to_script_json(obj):
    curr = None
    if os.path.exists(f'{HOST_SCRIPTS_DIR}/scripts.json'):
        with open(f'{HOST_SCRIPTS_DIR}/scripts.json', 'r') as f:
            curr = json.load(f)
            curr.append(obj)
    else:
        curr = [obj]
    
    with open(f'{HOST_SCRIPTS_DIR}/scripts.json', 'w') as f:
        json.dump(curr, f)

def get_script_data_from_json():
    if os.path.exists(f'{HOST_SCRIPTS_DIR}/scripts.json'):
        with open(f'{HOST_SCRIPTS_DIR}/scripts.json', 'r') as f:
            return json.load(f)
    return []

def debian_poweroff():
    try:
        run_command(f'VBoxManage controlvm {VM_NAME} poweroff')
        time.sleep(1) # Wait for VM to power off
    except:
        pass # VM is not running

def debian_exec_script (directory, script_name, fetch_env_vars=None, arguments='', client=None):
    if not os.path.exists(f'{HOST_SCRIPTS_DIR}/{script_name}.sh'):
        raise Exception(f'Script {script_name}.sh does not exist in {HOST_SCRIPTS_DIR}')

    if fetch_env_vars:
        return debian_exec (f'cd {directory}', 
                    f'source {DEBIAN_VM_SCRIPTS_DIR}/{script_name}.sh {arguments}',
                    *[f'echo "{var}: ${var}"' for var in fetch_env_vars],
                    client=client,
                    raise_errors=False)
    else:
        return debian_exec (f'cd {directory}', 
                    f'bash {DEBIAN_VM_SCRIPTS_DIR}/{script_name}.sh {arguments}',
                    client=client,
                    raise_errors=False)

def debian_checkpoint (checkpoint_name):
    try:
        # Delete the checkpoint if it already exists
        run_command(f'VBoxManage snapshot {VM_NAME} delete {checkpoint_name}')
    except:
        pass
    run_command(f'VBoxManage snapshot {VM_NAME} take {checkpoint_name}')

def debian_startup():
    run_command(f'VBoxManage startvm "{VM_NAME}" --type headless')
    time.sleep(7)


def debian_checkpoint_reset (checkpoint_name):
    debian_poweroff()
    run_command(f'VBoxManage snapshot {VM_NAME} restore {checkpoint_name}')
    debian_startup()
    
def debian_install_utilities():
    # Add trivy repository
    debian_exec('wget -qO - https://aquasecurity.github.io/trivy-repo/deb/public.key | sudo apt-key add -',
                'echo deb https://aquasecurity.github.io/trivy-repo/deb $(lsb_release --sc) main | sudo tee -a /etc/apt/sources.list.d/trivy.list')
    
    # Update repositories
    debian_exec('apt-get update')

    # Install utilities
    for utility in STARTING_UTILITIES:
        debian_exec(f'DEBIAN_FRONTEND="noninteractive" apt-get install -y {utility}')

    # install docker
    debian_exec('sudo install -m 0755 -d /etc/apt/keyrings',
                'curl -fsSL https://download.docker.com/linux/debian/gpg | sudo tee /etc/apt/keyrings/docker.asc > /dev/null',
                'sudo chmod a+r /etc/apt/keyrings/docker.asc',
                'echo "deb [arch=amd64,arm64,ppc64el,s390x, signed-by=/etc/apt/keyrings/docker.asc] https://download.docker.com/linux/debian $(lsb_release -cs) stable" | sudo tee /etc/apt/sources.list.d/docker.list > /dev/null',
                'sudo apt update',
                'sudo apt install -y docker-ce docker-ce-cli containerd.io docker-buildx-plugin docker-compose-plugin',
                'systemctl stop docker')
    
    debian_exec(f'apt-file update') # For automatic installation of missing utilities used in scripts

def docker_exec (container, command):
    return run_command(f'docker exec {container} {command}')

def shellcheck_exec (command):
    return docker_exec('shellcheck', f'bash -c "{command}"')

def shellcheck_copy_scripts():
    return run_command(f'docker cp {HOST_SCRIPTS_DIR}/. shellcheck:{SHELLCHECK_VM_SCRIPTS_DIR}')

def shellcheckRunOnScript (script_name):
    if '#!' in read_script(script_name):
        res = shellcheck_exec(f'shellcheck --format=json {SHELLCHECK_VM_SCRIPTS_DIR}/{script_name}.sh').stdout
    else:
        res = shellcheck_exec(f'shellcheck --format=json <(echo \\"#!/bin/bash\\n\\"; cat {SHELLCHECK_VM_SCRIPTS_DIR}/{script_name}.sh)').stdout
    return json.loads(res)
def postgresExec (command):
    return docker_exec('postgres_db', command)

def saveResultRow (
    experiment_name,
    problem_name,
    problem_desc,
    problem_category,
    problem_complexity,
    attempt,
    generated_code,
    temperature,
    topp,
    model_name,
    case_name,
    case_complexity,
    error,
    is_correct,
    result,
    sh_style_cnt,
    sh_info_cnt,
    sh_warning_cnt,
    sh_error_cnt,
    sh_output,
    score,
    tokens_used,
    ms_to_generate,
    mistake_reason,
    type
):
    query = f"""
    INSERT INTO ExperimentResults (
        experiment_name, 
        problem_name,
        problem_desc, 
        problem_category, 
        problem_complexity, 
        attempt, 
        generated_code, 
        temperature, 
        topp, 
        model_name, 
        case_name, 
        case_complexity, 
        error, 
        is_correct, 
        result, 
        sh_style_cnt, 
        sh_info_cnt, 
        sh_warning_cnt, 
        sh_error_cnt, 
        sh_output, 
        score,
        tokens_used,
        ms_to_generate,
        mistake_reason,
        type
    ) VALUES (
        '{experiment_name.replace("'", "''")}', 
        '{problem_name.replace("'", "''")}',
        '{problem_desc.replace("'", "''")}',
        '{problem_category.replace("'", "''")}', 
        {problem_complexity}, 
        {attempt}, 
        '{generated_code.replace("'", "''")}', 
        {temperature}, 
        {topp}, 
        '{model_name.replace("'", "''")}', 
        '{case_name.replace("'", "''")}', 
        {case_complexity}, 
        '{error[-500:].replace("'", "''")}', 
        {is_correct}, 
        '{result.replace("'", "''")}', 
        {sh_style_cnt}, 
        {sh_info_cnt}, 
        {sh_warning_cnt}, 
        {sh_error_cnt}, 
        '{sh_output[-500:].replace("'", "''")}', 
        {score},
        {tokens_used},
        {ms_to_generate},
        '{mistake_reason.replace("'", "''")}',
        '{type.replace("'", "''")}'
    );
    """.replace('\n', ' ').replace('    ', ' ').replace('  ', ' ').replace('"', '*')
    try:
        postgresExec(f'psql -U superset_user -d superset_db -c "{query}"')
    except Exception as e:
        print(query)
        raise e

def clearProblem(experiment_name, problem_name):
    query = f"""
    DELETE FROM ExperimentResults
    WHERE experiment_name = '{experiment_name}' AND problem_name = '{problem_name}';
    """.replace('\n', ' ').replace('    ', ' ').replace('  ', ' ')
    postgresExec(f'psql -U superset_user -d superset_db -c "{query}"')

def resultExists(experiment_name, problem_name, model_name, topp, temperature, attempt):
    query = f"""
    SELECT COUNT(*) FROM ExperimentResults
    WHERE experiment_name = '{experiment_name}' AND problem_name = '{problem_name}' AND model_name = '{model_name}' AND topp = {topp} AND temperature = {temperature} AND attempt = {attempt};
    """.replace('\n', ' ').replace('    ', ' ').replace('  ', ' ')
    return int(postgresExec(f'psql -U superset_user -d superset_db -t -c "{query}"').stdout.strip()) > 0

def setup_vbox_api():
    print('Starting virtualbox http server to access the SOAP API...')
    def start_vbox_server():
        run_command('vboxwebsrv -H 127.0.0.1 -v')
    vbox_server_thread = threading.Thread(target=start_vbox_server, daemon=True)
    vbox_server_thread.start()

def replace_api_mirror(model_name):
    oldIndex = None
    for i, model in enumerate(MODELS):
        if model['name'] == model_name:
            oldIndex = i
            break
    
    if oldIndex == None:
        raise Exception(f'Model {model_name} not found')
    
    for i, mirror in enumerate(MODEL_MIRRORS):
        if mirror['name'] == model_name:
            temp = MODELS[oldIndex]
            MODELS[oldIndex] = mirror
            MODEL_MIRRORS[i] = temp
            return mirror
    
    raise Exception(f'Mirror for model {model_name} not found')

<>:138: SyntaxWarning: invalid escape sequence '\.'
<>:138: SyntaxWarning: invalid escape sequence '\.'
C:\Users\Banknote\AppData\Local\Temp\ipykernel_185348\2097804381.py:138: SyntaxWarning: invalid escape sequence '\.'
  ip = re.search("'([0-9\.]+)'", run_command(f'VBoxManage guestproperty enumerate "debian_vbox" | {grep_utility} IP').stdout).group(1)


# Setup external tools
### Requirements:
1. Installed VirtualBox (scripted was tested with version 7.1.6)
2. Installed and running Docker Desktop (script was tested with version 4.37.0)
3. Internet connection
4. ~15GB free space
5. Windows OS (though only small changes are needed to adapt the code to be run on a linux/mac system)

### Setup Debian VM in VirtualBox:

In [ ]:
# Create the directory if it does not exist
os.makedirs(ISO_PATH, exist_ok=True)

if not os.path.isfile(f'{ISO_PATH}/{VM_NAME}.iso'):
    print('Downloading Debian ISO...')
    urllib.request.urlretrieve(DEBIAN_ISO, f'{ISO_PATH}/{VM_NAME}.iso')
    print('Download complete!')
else:
    print('Debian ISO already downloaded, skipping')

if not os.path.isfile(f'{ISO_PATH}/VBoxGuestAdditions.iso'):
    print('Downloading VirtualBox Guest Additions ISO...')
    urllib.request.urlretrieve(GUEST_ADDITIONS_ISO_URL, f'{ISO_PATH}/VBoxGuestAdditions.iso')
    print('Download complete!')
else:
    print('VirtualBox Guest Additions ISO already downloaded, skipping')

In [ ]:
if FIRST_TIME_SETUP:
    def vm_exists(vm_name):
        vms = run_command('VBoxManage list vms').stdout
        return f'"{vm_name}"' in vms  # VBoxManage encloses VM names in quotes

    if vm_exists(VM_NAME):
        # Delete the VM if it already exists
        debian_poweroff()
        run_command(f'VBoxManage unregistervm {VM_NAME} --delete')
    # Create a new VM
    run_command(f'VBoxManage createvm --name "{VM_NAME}" --ostype Debian_64 --register')

    # Set VM settings (VM specs may be adjusted as needed, though lowering them has been observed to cause instability in the VM)
    run_command(f'VBoxManage modifyvm "{VM_NAME}" --memory 8192 --cpus 7 --vram 64 --graphicscontroller VMSVGA')

    # Create virtual hard disk (~15GB)
    os.makedirs(f'{VM_FOLDER}', exist_ok=True)
    run_command(f'VBoxManage createmedium disk --filename "{VM_FOLDER}/{VM_NAME}.vdi" --size 15000')

    # Attach storage controller
    run_command(f'VBoxManage storagectl "{VM_NAME}" --name "SATA Controller" --add sata --controller IntelAhci')

    # Attach hard disk
    run_command(f'VBoxManage storageattach "{VM_NAME}" --storagectl "SATA Controller" --port 0 --device 0 --type hdd --medium "{VM_FOLDER}/{VM_NAME}.vdi"')

    # Attach Debian ISO
    run_command(f'VBoxManage storagectl "{VM_NAME}" --name "IDE Controller" --add ide')
    run_command(f'VBoxManage storageattach "{VM_NAME}" --storagectl "IDE Controller" --port 1 --device 0 --type dvddrive --medium "{ISO_PATH}/{VM_NAME}.iso"')

    # Add a second sata controller for guest additions iso (because IDE Controller is already used)
    run_command(f'VBoxManage storageattach "{VM_NAME}" --storagectl "SATA Controller" --port 1 --device 0 --type dvddrive --medium "{ISO_PATH}/VBoxGuestAdditions.iso"')

    # disable automatic time sync to avoid issues for problems that require specific time
    run_command(f'VBoxManage setextradata "{VM_NAME}" "VBoxInternal/Devices/VMMDev/0/Config/GetHostTimeDisabled" 1')

    run_command(f'VBoxManage modifyvm "{VM_NAME}" --biosbootmenu disabled')

    # Add scripts as shared folder
    if not os.path.exists(SHARED_FOLDER):
        os.makedirs(SHARED_FOLDER)
    run_command(f'VBoxManage sharedfolder add "{VM_NAME}" --name "scripts" --hostpath "{SHARED_FOLDER}" --automount')

    # Set only one network adapter for the NAT connection
    def get_network_adapter_full_name(interface):
        # Retrieve the full name of a network adapter.
        if platform.system() == "Windows" and win32com:
            objWMIService = win32com.client.Dispatch("WbemScripting.SWbemLocator").ConnectServer(".", "root\cimv2")
            adapters = objWMIService.ExecQuery("SELECT * FROM Win32_NetworkAdapter")
            for adapter in adapters:
                if adapter.NetConnectionID == interface:
                    return adapter.Name
        return interface  # Fallback to the interface name if lookup fails

    def get_active_network_adapter():
        # Find a network adapter that has internet access
        for interface, addrs in psutil.net_if_addrs().items():
            for addr in addrs:
                if addr.family == socket.AF_INET:  # IPv4 addresses only
                    try:
                        # Create a socket using this adapter to test connectivity
                        test_socket = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
                        test_socket.bind((addr.address, 0))
                        test_socket.connect(("8.8.8.8", 80))
                        test_socket.close()
                        return get_network_adapter_full_name(interface)  # Return full adapter name
                    except Exception:
                        pass
        return None

    active_adapter = get_active_network_adapter()
    if active_adapter == None:
        raise "Network connected adapter not found!"

    run_command(f'VBoxManage modifyvm "{VM_NAME}" --nic1 bridged')
    run_command(f'VBoxManage modifyvm "{VM_NAME}" --bridgeadapter1 "{active_adapter}"')

    # Start the VM
    print("Starting VM...")
    debian_startup()
    time.sleep(7) # Make sure the boot menu has time to open
    print('Debian started. Installing...')
    
    def debian_enter_input(command):
        # Enter input in the VM (as if from keyboard), useful before ssh has been set up
        # VirtualBox uses keyboard scancode set 1 (https://users.utcluj.ro/~baruch/sie/labor/PS2/Scan_Codes_Set_1.htm)
        def get_text_hex(scancode):
            text = hex(scancode)[2:].lower()
            if len(text) % 2 == 1:
                text = '0' + text
            if len(text) > 2:
                text = ' '.join([text[i:i+2] for i in range(0, len(text), 2)])
            return text
        
        scan_codes = []
        for char in command:
            scan_code_numeric = keyboard.key_to_scan_codes(char.lower())[0]

            needs_shift = False
            if char.isupper() or char in '~!@#$%^&*()_+{}|:"<>?':
                scan_codes.append('2a') # Press left shift
                needs_shift = True

            scan_codes.append(get_text_hex(scan_code_numeric))
            scan_codes.append(get_text_hex(scan_code_numeric | 0x80)) # Release key

            if needs_shift:
                scan_codes.append('aa') # Release left shift

        run_command(f'VBoxManage controlvm {VM_NAME} keyboardputscancode {" ".join(scan_codes)}')
        run_command(f'VBoxManage controlvm {VM_NAME} keyboardputscancode 1c 9c') # Click Enter

    def navigate_down(times):
        for _ in range(times):
            run_command(f'VBoxManage controlvm {VM_NAME} keyboardputscancode e0 50 e0 d0') # Click down
            time.sleep(0.2)
    
    # Navigate "Advanced options"
    navigate_down(2)
    run_command(f'VBoxManage controlvm {VM_NAME} keyboardputscancode 1c 9c') # Click Enter

    # Navigate "Automated install"
    navigate_down(6)
    run_command(f'VBoxManage controlvm {VM_NAME} keyboardputscancode 1c 9c') # Click Enter

    # Start local http server to host the preseed file

    # Get host address in local network
    def get_ipv4_address():
        with socket.socket(socket.AF_INET, socket.SOCK_DGRAM) as s:
            s.connect(("8.8.8.8", 80))  # Connect to an external server
            return s.getsockname()[0]
    host_ip = get_ipv4_address()

    # Start server on a background thread since server_forever is synchronous
    server = HTTPServer(("0.0.0.0", PRESEED_PORT), http.server.SimpleHTTPRequestHandler)
    current_dir = os.getcwd()
    os.chdir(PRESEED_FOLDER)
    try:
        def start_preseed_http_server():
            server.serve_forever()
        server_thread = threading.Thread(target=start_preseed_http_server, daemon=True)
        server_thread.start()

        time.sleep(100) # Wait for preseed prompt to load
        debian_enter_input(f'http://{host_ip}:{PRESEED_PORT}/preseed.cfg') # Virtual box host address IP
        time.sleep(600) # Wait for the installation
    except Exception as e:
        server.shutdown()
        os.chdir(current_dir)
        raise e
    server.shutdown()
    os.chdir(current_dir)

    print('OS installed! Installing utilities...')
    debian_install_utilities()
    debian_checkpoint(START_CHECKPOINT)
else:
    print('Restoring VM checkpoint...')
    debian_checkpoint_reset(START_CHECKPOINT)
    
print('VM is ready to use!')

### Setup the dockerized tools (ShellCheck, Postgres DB and Apache Superset):

In [ ]:
%%cmd
# This won't delete data since it's kept on host (via Docker's volumes)
docker compose down
docker compose build
docker compose up -d

# Script generation implementation

In [5]:
def extract_bash_code(text):
    pattern = re.compile(r'(?:bash|sh)```(.*?)```', re.DOTALL)
    matches1 = re.finditer(pattern, text)
    pattern = re.compile(r'```(?:bash|sh)\n(.*?)```', re.DOTALL)
    matches2 = re.finditer(pattern, text)

    results = [s[1].strip('\n') for s in sorted([(match.start(), match.group(1)) for match in matches1] + [(match.start(), match.group(1)) for match in matches2])]
    ret = "\n".join(results)

    # If no code marked with bash or sh is found try to extract all code blocks
    if len(ret) == 0:	
        pattern = re.compile(r'```(.*?)```', re.DOTALL)
        matches = pattern.findall(text)
        ret = "\n".join([match.strip('\n') for match in matches])
    
    return ret

'''
previous_conversation = [
    {
        'ai_answer': 'string',
        'issues': 'string'
    },
    ...
]
'''
def generate_script(problem_text, model, temp, top_p, previous_conversation, use_mirrors=True):
    model_is_o1 = 'o1' in model['name']

    # o1 doesn't support system role
    system_role = 'system' if not model_is_o1 else 'user'
    
    messages = [
        {
            'role': system_role, 
            'content': SYSTEM_CONTEXT
        },
        {
            'role': 'user',
            'content': problem_text
        },
    ]

    for prev in previous_conversation:
        messages.append({
            'role': 'assistant',
            'content': prev['ai_answer']
        })
        
        messages.append({
            'role': 'user',
            'content': prev['issues']
        })

    class try_generate_script_thread(threading.Thread):
        def __init__(self, bucket, model, temp, top_p, messages, timeout, client):
            threading.Thread.__init__(self, daemon=True)
            self.bucket = bucket
            self.model = model
            self.temp = temp
            self.top_p = top_p
            self.messages = messages
            self.timeout = timeout
            self.client = client
            
        def run(self):
            try:
                # Technical name is used for the mirror models if the API expects a different name in order to keep the actual model name the same throughout the experiment
                techninal_name = self.model['techninal_name'] if 'techninal_name' in self.model else self.model['name']
                completion = self.client.chat.completions.create(
                    model=techninal_name,
                    messages=self.messages,
                    temperature=self.temp,
                    top_p=self.top_p,
                    frequency_penalty=0,
                    presence_penalty=0,
                    timeout=self.timeout,
                )

            except Exception as e:
                self.bucket.put(e)
                return

            if isinstance(completion, list) and 'error' in completion[0].model_extra:
                self.bucket.put(Exception(f'Error generating script: {completion[0].model_etxra["error"]}'))
                return

            self.bucket.put(completion)

    attempts_remaining = 3

    exception = None
    completion = None
    while True:
        bucket = queue.Queue()
        timeout = math.floor(80 * 1.2**(3-attempts_remaining)) * (2 if model['name'] == 'deepseek-r1' else 1) 
        client = OpenAI(api_key=model['key'], base_url=model['url'])
        generate_thread = try_generate_script_thread(bucket, model, temp, top_p, messages, timeout, client)
        start_time = time.time()
        generate_thread.start()
        while True:
            try:
                result = bucket.get(block=False)
                if isinstance(result, Exception):
                    exception = result
                    attempts_remaining -= 1
                    break
                completion = result
                break
            except queue.Empty:
                pass
            except Exception as e:
                raise e
            if not generate_thread.is_alive():
                attempts_remaining -= 1
                break
            if time.time() - start_time > timeout + 5:
                attempts_remaining -= 1
                client.close()
                generate_thread.join()
                break
                
            time.sleep(0.1)
        if completion != None:
            break
        elif attempts_remaining == 0:
            break
        else:
            time.sleep(10*(3-attempts_remaining))

    if completion == None:
        if exception != None:
            print(f'Failed to generate script: {model} {messages}')
            raise exception
        
        if use_mirrors:
            try:
                model = replace_api_mirror(model['name'])
            except:
                raise Exception(f'Timed out to generate script: {model} {messages}')
            
            return generate_script(problem_text, model, temp, top_p, previous_conversation, use_mirrors=False)
        else:
            raise Exception(f'Timed out to generate script: {model} {messages}')


    tokens_used = completion.usage.completion_tokens

    # Get rid of formatting around the script
    # If no bash script is found, return NOT_A_BASH
    generated_code = extract_bash_code(completion.choices[0].message.content)
    if len(generated_code) == 0:
        return NOT_A_BASH, tokens_used
    else:
        return generated_code, tokens_used


# Generate all scripts required for the experiment
def generate_scripts_all(problem_text):
    results = []
    for model in MODELS:
        model_is_o1 = 'o1' in model['name']

        if not model_is_o1:
            for params in HYPERPARAMERTRIZATIONS:
                top_p = params['top_p']
                temp = params['temperature']
                # Check if the script is already generated
                script_found = False
                for script in SCRIPTS_ALREADY_GENERATED:
                    if script['problem_text'] == problem_text and script['model'] == model['name'] and script['temperature'] == temp and script['top_p'] == top_p:
                        text = read_script(script['script_name'])
                        if text != None:
                            script_found = True
                            results.append({
                                'model': model['name'],
                                'script': text,
                                'tokens_used': script['tokens_used'],
                                'top-p': top_p,
                                'time_taken_ms': script['time_taken_ms'],
                                'temperature': temp,
                                'loaded_from_cache': True
                            })
                            break
                if script_found:
                    continue
                
                start_time = time.time()
                result, tokens_used = generate_script(problem_text, model, temp, top_p, [])
                time_elapsed = time.time() - start_time

                results.append({
                    'model': model['name'],
                    'temperature': temp,
                    'top-p': top_p,
                    'script': result,
                    'time_taken_ms': time_elapsed * 1000,
                    'tokens_used': tokens_used,
                    'loaded_from_cache': False
                })
        else:
            # Check if the script is already generated
            script_found = False
            for script in SCRIPTS_ALREADY_GENERATED:
                if script['problem_text'] == problem_text and script['model'] == model['name'] and script['temperature'] == 1 and script['top_p'] == 1:
                    text = read_script(script['script_name'])
                    if text != None:
                        script_found = True
                        results.append({
                            'model': model['name'],
                            'script': text,
                            'tokens_used': script['tokens_used'],
                            'temperature': 1,
                            'top-p': 1,
                            'time_taken_ms': script['time_taken_ms'],
                            'loaded_from_cache': True
                        })
                        break
            if script_found:
                continue

            # o1 doesn't support temperature and top-p besides 1
            start_time = time.time()
            result, tokens_used = generate_script(problem_text, model, 1, 1, [])
            time_elapsed = time.time() - start_time

            results.append({
                'model': model['name'],
                'temperature': 1,
                'top-p': 1,
                'script': result,
                'time_taken_ms': time_elapsed * 1000,
                'tokens_used': tokens_used,
                'loaded_from_cache': False
            })
    return results


# Testing implementation

In [6]:
class Problem(ABC):

    # Implementē katram uzdevumam
    name = None
    category = None
    level = None
    text = None
    type = None # EDIT/OUTPUT
    PROBLEM_CHECKPOINT = None

    test_cases = []

    def __init__(self, test_cases):
        self.test_cases = test_cases

    # Generated scripts
    scripts = []

    # Rezultāti
    results = dict()
    '''
    { 
        script_name: {
            test_cases: {
                {test_case}: {
                    test_name: 'string',
                    test_level: 'string',
                    functional_executed: bool,
                    functional_test_success: bool,
                    static_clean: bool,
                    functional_error: 'string',
                    functional_issue: 'string',
                    output: 'string',
                }
            },
            static_issues: 'string',
            sh_style_cnt: int,
            sh_info_cnt: int,
            sh_warning_cnt: int,
            sh_error_cnt: int,
        },
        ...
    }
    '''

    # Ģenerē visus skriptus pirmajam mēģinājumam, saglabā tos objektā
    def generateScripts(self):
        self.scripts = []
        scripts = generate_scripts_all(self.text)
        for script in scripts:
            script_name = f'{self.name}_{script["model"]}_{script["temperature"]}_{script["top-p"]}'.replace("/", "_")
            script["name"] = script_name
            self.scripts.append(script)
            if not script['loaded_from_cache']:
                save_script(script_name, script['script'])
                append_to_script_json({
                    'problem_text': self.text,
                    'model': script['model'],
                    'temperature': script['temperature'],
                    'top_p': script['top-p'],
                    'script_name': script_name,
                    'time_taken_ms': script['time_taken_ms'],
                    'tokens_used': script['tokens_used']
                })

    # Otreizēji ģenerē konkrētas parametrizācijas skriptu padodot iepriekšējo skriptu un problēmu, atgriež skripta nosaukumu
    def regenerateScript(self, model, temperature, top_p, issue):
        original_name = f'{self.name}_{model}_{temperature}_{top_p}'.replace("/", "_")
        original_content = read_script(original_name)

        model_params = None
        for m in MODELS:
            if m['name'] == model:
                model_params = m
                break

        start_time = time.time()
        new_script_content, tokens_used = generate_script(self.text, model_params, temperature, top_p, [{'ai_answer': f'```bash\n{original_content}\n```', 'issues': issue}])
        time_elapsed = time.time() - start_time
        
        new_name = f'{original_name}_2'
        save_script(new_name, new_script_content)
        return {
                'model': model,
                'name': new_name,
                'temperature': temperature,
                'top-p': top_p,
                'script': new_script_content,
                'time_taken_ms': time_elapsed * 1000,
                'tokens_used': tokens_used,
                'loaded_from_cache': False
        }

    def reset_checkpoint(self):
        debian_checkpoint_reset(START_CHECKPOINT)
        self.init_problem()

    @abstractmethod
    def init_problem(self):   
        '''
        Uzstāda uzdevuma pamata konfigurāciju
        '''
        pass

    def run_static(self, script_name):
        '''
        1. Pārbauda skriptu ar statiskiem ShellCheck testiem
        2. Saglabā rezultātus objektā
    
        Piemēr ShellCheck izvadei:
        [
            {
                "file": "usr/scripts/example.sh",
                "line": 1,
                "endLine": 1,
                "column": 12,
                "endColumn": 12,
                "level": "error",
                "code": 1017,
                "message": "Literal carriage return. Run script through tr -d '\\r' .",
                "fix": null
            },
            ...
        ]
        '''
        result = shellcheckRunOnScript(script_name)

        # Only unique issues are counted
        unique_static_issues_dict = {}
        for issue in result:
            unique_static_issues_dict[issue['code']] = issue

        self.results[script_name]['sh_style_cnt'] = len([issue for issue in unique_static_issues_dict.values() if issue['level'] == 'style'])
        self.results[script_name]['sh_info_cnt'] = len([issue for issue in unique_static_issues_dict.values() if issue['level'] == 'info'])
        self.results[script_name]['sh_warning_cnt'] = len([issue for issue in unique_static_issues_dict.values() if issue['level'] == 'warning'])
        self.results[script_name]['sh_error_cnt'] = len([issue for issue in unique_static_issues_dict.values() if issue['level'] == 'error'])

        if len([issue for issue in unique_static_issues_dict.values()]) > 0:
            self.results[script_name]['static_clean'] = False
        else:
            self.results[script_name]['static_clean'] = True

        # Formatē static_issues teksstu no JSON rezultātiem modelim saprotamā veidā
        self.results[script_name]['static_issues'] = ''
        for issue in result:
            if issue['line'] == issue['endLine']:
                self.results[script_name]['static_issues'] += f"Line {issue['line']}: {issue['message']}\n"
            else:
                self.results[script_name]['static_issues'] += f"Lines {issue['line']} - {issue['endLine']}: {issue['message']}\n"

    # Palaiž testus padotajam skriptam, atgriež rezultātus
    def run_test_cases(self, script_name, skip_static=False):
        self.results[script_name] = {'test_cases': dict()}
        if not skip_static:
            self.run_static(script_name)

        for test_case in self.test_cases:
            test_case.run(script_name, self)

        return self.results[script_name]

class TestCase(ABC):
    
    # Šo jādefinē katrai mantojošajai klasei
    name = None
    level = None
    CHECKPOINT_NAME = None

    def run(self, script_name, problem, skip_checkpoint=False, packages_tried=[]):
        if not skip_checkpoint:
            problem.reset_checkpoint()
        self.setup_functional(problem)
        self.run_functional_internal(problem, script_name)

        # If the issue was due to a utility not being installed, install it and rerun the test
        if not problem.results[script_name]['test_cases'][self.name]['functional_test_success'] and 'command not found' in problem.results[script_name]['test_cases'][self.name]['functional_error']:
            debian_checkpoint_reset(START_CHECKPOINT)
            missing_utility = re.search(r"([A-Za-z0-9\-\_]+): command not found", problem.results[script_name]['test_cases'][self.name]['functional_error']).group(1)
            try:
                apt_file_output = debian_exec(f'apt-file search --regexp "/{missing_utility}$"').stdout # Only search for executables with the exact name
            except:
                problem.results[script_name]['test_cases'][self.name]['functional_issue'] = f'The script attempted to use a utility {missing_utility} that is not available in the Debian repositories (via apt-get install)'
                return
            package_found = False
            
            # Since often the utility has the same name as the package apt-file search can return multiple pacakges, prioritize the one that has the utility name in the beginning
            if f'{missing_utility}:' in apt_file_output:
                package = missing_utility
                package_found = True
            else:
                pattern = re.compile(r"^([A-Za-z0-9\-\_]+):", re.MULTILINE)
                for match in pattern.finditer(apt_file_output):
                    package = match.group(1)
                    if package not in packages_tried:
                        package_found = True
                        break
            
            if not package_found:
                problem.results[script_name]['test_cases'][self.name]['functional_issue'] = f'The script attempted to use a utility {missing_utility} that is not available in the Debian repositories (via apt-get install)'
                return

            print(f'Installing {package}...')
            res = debian_exec(f'DEBIAN_FRONTEND="noninteractive" apt-get install -y {package}')
            debian_checkpoint(START_CHECKPOINT) # Save the checkpoint in case other tests also require the utility

            packages_tried.append(package)
            problem.init_problem()
            self.run(script_name, problem, skip_checkpoint=True, packages_tried=packages_tried) # rerun the test

    @abstractmethod
    def setup_functional(self, problem):
        '''
        Uzstāda testa konfigurāciju
        '''

    @abstractmethod
    def run_functional_internal(self, problem, script_name):
        '''
        1. Darbina skriptu ar nosaukumu script_name
        2. Pārbauda rezultātus un saglabā tos objektā (jāuzstāda functional_executed, functional_test_success, functional_error, functional_issue)
        '''
        pass
    

def get_check_command(condition):
    return f'if [ {condition} ]; then echo "YES"; else echo "NO"; fi'

# Experiment problems

In [7]:
problems = []

def test_exec(test, problem, script_name, find_issue_func, exec_dir, fetch_env_vars=None, arguments=None):
        result_dict = dict()

        result_dict['test_name'] = test.name
        result_dict['test_level'] = test.level
        
        TIMEOUT = 400
        client = create_ssh_connection('root', 'preseed')

        class try_exec(threading.Thread):
            def __init__(self, bucket, script_name, fetch_env_vars, arguments, client):
                threading.Thread.__init__(self, daemon=True)
                self.bucket = bucket
                self.script_name = script_name
                self.fetch_env_vars = fetch_env_vars
                self.arguments = arguments
                self.client = client

            def run(self):
                results = debian_exec_script(exec_dir, script_name, fetch_env_vars=fetch_env_vars, arguments=arguments, client=self.client)
                self.bucket.put(results)

        bucket = queue.Queue()
        exec_thread = try_exec(bucket, script_name, fetch_env_vars, arguments, client)
        exec_thread.start()
        start_time = time.time()

        timed_out = False

        while True:
            time.sleep(1)
            try:
                results = bucket.get(block=False)
                break
            except queue.Empty:
                pass
            if not exec_thread.is_alive():
                break
            if time.time() - start_time > TIMEOUT:
                    client.close()
                    exec_thread.join()
                    result_dict['functional_issue'] = 'Script won\'t finish'
                    result_dict['functional_issue_type'] = 'CONDITION_NOT_MET'
                    result_dict['functional_test_success'] = False
                    result_dict['functional_error'] = ''
                    result_dict['output'] = ''
                    timed_out = True
                    break

        # Check results
        if not timed_out:
            issue, issue_type = find_issue_func(test, problem, results)
            if issue:
                result_dict['functional_error'] = results.stderr
                result_dict['functional_issue'] = issue
                result_dict['functional_issue_type'] = issue_type
                result_dict['output'] = results.stdout
                result_dict['functional_test_success'] = False
            else:
                result_dict['functional_test_success'] = True

        problem.results[script_name]['test_cases'][test.name] = result_dict

'''
Kategorija: Ikdienas uzdevumu automatizācija 
Sarežģītības līmenis: 1
Uzdevuma teksts: Delete all the image files in the current directory and it’s subdirectories
Testa gadījumi 
    1. (1. līmenis): Attēli ar populārāriem paplašinājumiem tajā pašā direktorijā
    2. (1. līmenis): Attēli ar populārāriem paplašinājumiem 1 apakšdirektorijā
    3. (2. līmenis): Attēli ar populārāriem paplašinājumiem sākotnējā direktorijā un vairāku līmeņu apakšdirektorijās katrā pa attēlam un vienai citai datnei
    4. (3. līmenis): Attēli ar nepopulārāriem paplašinājumiem sākotnējā direktorijā un vairāku līmeņu apakšdirektorijās katrā pa attēlam un vienai citai datnei
    5. (3. līmenis): 1 attēls ar paplašinājumu “JPG” (Debian noklusējuma ext4 failsistēmā paplašinājumi ir reģistrjūtīgi) un viena direktorija ar attēla paplašinājumu nosaukumā
'''

class DeleteImages(Problem):
    name = 'DeleteImages'
    category = 'Ikdienas uzdevumu automatizācija'
    level = 1
    text = 'Delete all the image files in the current directory and it’s subdirectories'
    type = 'RISK'

    image_dir = '/usr/images_to_delete'

    def init_problem(self):
        debian_exec(f'mkdir -p {self.image_dir}')

class TestCaseDeleteImages(TestCase):
    name = 'Attēli ar populārāriem paplašinājumiem tajā pašā direktorijā'
    level = 1

    def setup_functional(self, problem):
        debian_exec(f'cd {problem.image_dir}',
                    'touch image1.jpg',
                    'touch image2.png',
                    'touch image3.jpeg'
                    )

    def run_functional_internal(self, problem, script_name):
        def find_issue(test, problem, results):
            try:
                out = debian_exec(f'cd {problem.image_dir}',
                                get_check_command('-e image1.jpg'),
                                get_check_command('-e image2.png'), 
                                get_check_command('-e image3.jpeg')
                                )
                if 'YES' in out.stdout.strip():
                    return 'One or more images were not deleted.', 'CONDITION_NOT_MET'
            except:
                return 'The directory was corrupted. Checking whether the images were deleted is not possible.', 'TOO_MUCH_DONE'
            return None, None
        test_exec(self, problem, script_name, find_issue, problem.image_dir)

class TestCaseDeleteImages2(TestCase):
    name = 'Attēli ar populārāriem paplašinājumiem 1 apakšdirektorijā'
    level = 1

    def setup_functional(self, problem):
        debian_exec(f'mkdir -p {problem.image_dir}/subdir',
                    f'cd {problem.image_dir}/subdir',
                    'touch image1.jpg',
                    'touch image2.png',
                    'touch image3.jpeg'
                    )

    def run_functional_internal(self, problem, script_name):
        def find_issue(test, problem, results):
            try:
                out = debian_exec(f'cd {problem.image_dir}/subdir',
                                get_check_command('-e image1.jpg'),
                                get_check_command('-e image2.png'), 
                                get_check_command('-e image3.jpeg')
                                )
                if 'YES' in out.stdout.strip():
                    return 'One or more images were not deleted.', 'CONDITION_NOT_MET'
            except:
                return 'The directory was corrupted. Checking whether the images were deleted is not possible.', 'TOO_MUCH_DONE'
            return None, None 
        test_exec(self, problem, script_name, find_issue, problem.image_dir)
    
class TestCaseDeleteImages3(TestCase):
    name = 'Attēli ar populārāriem paplašinājumiem sākotnējā direktorijā un vairāku līmeņu apakšdirektorijās katrā pa attēlam un vienai citai datnei'
    level = 2

    def setup_functional(self, problem):
        debian_exec(f'cd {problem.image_dir}',
                    'touch image1.jpg',
                    'touch text1.txt',
                    'mkdir -p subdir1',
                    'cd subdir1',
                    'touch image2.png',
                    'touch text2.txt',
                    'cd ..',
                    'mkdir -p subdir2',
                    'cd subdir2',
                    'touch image3.jpeg',
                    'touch text3.txt',
                    'mkdir -p subdir3',
                    'cd subdir3',
                    'touch image4.jpg',
                    'touch text4.json' 
                    )

    def run_functional_internal(self, problem, script_name):
        def find_issue(test, problem, results):
            try:
                out = debian_exec(f'cd {problem.image_dir}',
                                  get_check_command('-e image1.jpg')
                                )
                if out.stdout.strip() != 'NO':
                    return "An image was not deleted", 'CONDITION_NOT_MET'
                
                out = debian_exec(f'cd {problem.image_dir}',
                                    get_check_command('-e text1.txt')
                                )
                if out.stdout.strip() != 'YES':
                    return "A non-image file was deleted", 'TOO_MUCH_DONE'

                subdirectories_not_deleted = False
                subdirectories_too_much_deleted = False

                out = debian_exec(f'cd {problem.image_dir}/subdir1',
                                    get_check_command('-e image2.png')
                                )

                if out.stdout.strip() != 'NO':
                    subdirectories_not_deleted = True
                
                out = debian_exec(f'cd {problem.image_dir}/subdir1',
                                    get_check_command('-e text2.txt')
                                )
                if out.stdout.strip() != 'YES':
                    subdirectories_too_much_deleted = True
                
                out = debian_exec(f'cd {problem.image_dir}/subdir2',
                                    get_check_command('-e image3.jpeg')
                                )
                if out.stdout.strip() != 'NO':
                    subdirectories_not_deleted = True
                
                out = debian_exec(f'cd {problem.image_dir}/subdir2',
                                    get_check_command('-e text3.txt')
                                )
                if out.stdout.strip() != 'YES':
                    subdirectories_too_much_deleted = True
                
                out = debian_exec(f'cd {problem.image_dir}/subdir2/subdir3',
                                    get_check_command('-e image4.jpg')
                                )
                if out.stdout.strip() != 'NO':
                    subdirectories_not_deleted = True
                
                out = debian_exec(f'cd {problem.image_dir}/subdir2/subdir3',
                                    get_check_command('-e text4.json')
                                )
                if out.stdout.strip() != 'YES':
                    subdirectories_too_much_deleted = True

                if subdirectories_not_deleted:
                    return "An image was not deleted in a subdirectory", 'CONDITION_NOT_MET'
                
                if subdirectories_too_much_deleted:
                    return "A non-image file was deleted in a subdirectory", 'TOO_MUCH_DONE'
            except:
                return 'The directory was corrupted. Checking whether the images were deleted is not possible.', 'TOO_MUCH_DONE'
            return None, None
        test_exec(self, problem, script_name, find_issue, problem.image_dir)

class TestCaseDeleteImages4(TestCase):
    name = 'Attēli ar nepopulārāriem paplašinājumiem sākotnējā direktorijā un vairāku līmeņu apakšdirektorijās katrā pa attēlam un vienai citai datnei'
    level = 3

    def setup_functional(self, problem):
        debian_exec(f'cd {problem.image_dir}',
                    'touch image1.apng',
                    'touch text1.txt',
                    'mkdir -p subdir1',
                    'cd subdir1',
                    'touch image2.avif',
                    'touch text2.txt',
                    'cd ..',
                    'mkdir -p subdir2',
                    'cd subdir2',
                    'touch image3.jfif',
                    'touch text3.txt',
                    'mkdir -p subdir3',
                    'cd subdir3',
                    'touch image4.webp',
                    'touch text4.json' 
                    )

    def run_functional_internal(self, problem, script_name):
        def find_issue(test, problem, results):
            try:
                out = debian_exec(f'cd {problem.image_dir}',
                                  get_check_command('-e image1.apng')
                                )
                if out.stdout.strip() != 'NO':
                    return "An image with extensions .apng was not deleted", 'CONDITION_NOT_MET'
                
                out = debian_exec(f'cd {problem.image_dir}',
                                    get_check_command('-e text1.txt')
                                )
                if out.stdout.strip() != 'YES':
                    return "A non-image file was deleted", 'TOO_MUCH_DONE'

                subdirectories_too_much_deleted = False

                out = debian_exec(f'cd {problem.image_dir}/subdir1',
                                    get_check_command('-e image2.avif')
                                )

                if out.stdout.strip() != 'NO':
                    return "An image with extensions .avif was not deleted", 'CONDITION_NOT_MET'
                
                out = debian_exec(f'cd {problem.image_dir}/subdir1',
                                    get_check_command('-e text2.txt')
                                )
                if out.stdout.strip() != 'YES':
                    subdirectories_too_much_deleted = True
                
                out = debian_exec(f'cd {problem.image_dir}/subdir2',
                                    get_check_command('-e image3.jfif')
                                )
                if out.stdout.strip() != 'NO':
                    return "An image with extensions .jfif was not deleted", 'CONDITION_NOT_MET'
                
                out = debian_exec(f'cd {problem.image_dir}/subdir2',
                                    get_check_command('-e text3.txt')
                                )
                if out.stdout.strip() != 'YES':
                    subdirectories_too_much_deleted = True
                
                out = debian_exec(f'cd {problem.image_dir}/subdir2/subdir3',
                                    get_check_command('-e image4.webp')
                                )
                if out.stdout.strip() != 'NO':
                    return "An image with extensions .webp was not deleted", 'CONDITION_NOT_MET'
                
                out = debian_exec(f'cd {problem.image_dir}/subdir2/subdir3',
                                    get_check_command('-e text4.json')
                                )
                if out.stdout.strip() != 'YES':
                    subdirectories_too_much_deleted = True
                
                if subdirectories_too_much_deleted:
                    return "A non-image file was deleted in a subdirectory", 'TOO_MUCH_DONE'
            except:
                return 'The directory was corrupted. Checking whether the images were deleted is not possible.', 'TOO_MUCH_DONE'
            return None, None
        test_exec(self, problem, script_name, find_issue, problem.image_dir)

class TestCaseDeleteImages5(TestCase):
    name = '1 attēls ar paplašinājumu “JPG” un 1 direktorija ar attēla paplašinājumu nosaukumā'
    level = 3

    def setup_functional(self, problem):
        debian_exec(f'cd {problem.image_dir}',
                    'touch image1.JPG',
                    'mkdir -p data.jpg'
                    )

    def run_functional_internal(self, problem, script_name):
        def find_issue(test, problem, results):
            try:
                out = debian_exec(f'cd {problem.image_dir}',
                                  get_check_command('-e image1.JPG')
                                )
                if out.stdout.strip() != 'YES':
                    return "An file with extension .JPG was deleted. The system uses default case-sensitive ext4 filesystem.", 'TOO_MUCH_DONE'
                
                out = debian_exec(f'cd {problem.image_dir}',
                                    get_check_command('-d data.jpg')
                                )
                if out.stdout.strip() != 'YES':
                    return 'A directory with the name "data.jpg" was deleted.', 'TOO_MUCH_DONE'  
            except:
                return 'The directory was corrupted. Checking whether the images were deleted is not possible.', 'TOO_MUCH_DONE'  
            return None, None
        test_exec(self, problem, script_name, find_issue, problem.image_dir)

DeleteImagesInstance = DeleteImages([
                                    TestCaseDeleteImages(),
                                    TestCaseDeleteImages2(), 
                                    TestCaseDeleteImages3(), 
                                    TestCaseDeleteImages4(),
                                    TestCaseDeleteImages5()
                                ])
problems.append(DeleteImagesInstance)

'''
Kategorija: Ikdienas uzdevumu automatizācija 
Sarežģītības līmenis: 3
Uzdevuma teksts: In the current directory, find json data in files with “user” keys and print out their corresponding values alphabetically (they will be strings) in the format “Here are the users: {user_1}, {user_2} … {user_n)”. If there are no users print “No users found”.
Testa gadījumi 
    1. (1. līmenis): Viena datne ar vienu korektu json, kur 3 lietotāji
    2. (1. līmenis): Viena datne ar vienu json, kur nav neviena lietotāja
    3. (2. līmenis): Divas json datnes ar lietotājiem abās un vēl viena papildus datne ar lietotājiem apakšdirektorijā (kuru nevajadzētu ņemt vērā)
    4. (2. līmenis): Viena datne, kur ir ievadīts “users”: “Tommy Black”, bet nav json, otra ar json, kur ir citi lietotāji                 
    5. (2. līmenis): Viena datne, kur ir vairāki json ar lietotājiem abos 
    6. (3. līmenis): Viena datne are 1 korektu json ar lietotājiem un otru nekorekta formāta, bet līdzīgu json (arī ar lietotājiem)
'''

class FindInJSON(Problem):
    name = 'FindInJSON'
    category = 'Ikdienas uzdevumu automatizācija'
    level = 3
    text = 'In the current directory, find json data in files with “user” keys and print out their corresponding values alphabetically (they will be strings) in the format “Here are the users: {user_1}, {user_2} … {user_n)”. If there are no users print “No users found”'
    type = 'NO_RISK'

    json_dir = '/usr/json_files'

    def init_problem(self):
        debian_exec(f'mkdir -p {self.json_dir}')

class TestCaseFindInJSON1(TestCase):
    name = 'Viena datne ar vienu korektu json, kur 3 lietotāji'
    level = 1

    json_data = """
        {
            "username": "Jack32",
            "user": "Jack Larson",
            "array": [1, 2, 3]
        }
    """

    def setup_functional(self, problem):
        debian_exec(f'cd {problem.json_dir}',
                    'touch data.json',
                    f"echo -e '{self.json_data.replace('\t', '\\t').replace('\n', '\\n')}' > data.json", 
                    )

    def run_functional_internal(self, problem, script_name):
        def find_issue(test, problem, results):
            if 'Here are the users: Jack Larson' not in results.stdout:
                if 'Jack' in results.stdout and 'Larson' in results.stdout:
                    return 'String with spaces outputted incorrectly.', 'CONDITION_PARTIALLY_MET'
                return 'A user was not found!', 'CONDITION_NOT_MET'
            return None, None
        test_exec(self, problem, script_name, find_issue, problem.json_dir)

class TestCaseFindInJSON2(TestCase):
    name = 'Viena datne ar vienu json, kur nav neviena lietotāja'
    level = 1

    json_data = """
        {
            "username": "Jack32",
            "array": [1, 2, 3]
        }
    """

    def setup_functional(self, problem):
        debian_exec(f'cd {problem.json_dir}',
                    'touch data.json',
                    f"echo -e '{self.json_data.replace('\t', '\\t').replace('\n', '\\n')}' > data.json", 
                    )

    def run_functional_internal(self, problem, script_name):
        def find_issue(test, problem, results):
            if 'No users found' not in results.stdout:
                return 'A user was found when there were none to be found!', 'CONDITION_NOT_MET'
            return None, None
        test_exec(self, problem, script_name, find_issue, problem.json_dir)

class TestCaseFindInJSON3(TestCase):
    name = 'Divas json datnes ar lietotājiem abās un vēl viena papildus datne ar lietotājiem apakšdirektorijā'
    level = 2

    json_data1 = """
        {
            "username": "JohnDoe",
            "user": "John Doe",
            "array": [1, 2, 3]
        }
    """

    json_data2 = """
        {
            "username": "Jack32",
            "user": "Jack Larson",
            "array": [1, 2, 3]
        }
    """

    json_data3 = """
        {
            "data": {
                "user": "Anne Page"
            }
        }
    """

    def setup_functional(self, problem):
        debian_exec(f'cd {problem.json_dir}',
                    'touch data1.json',
                    f"echo -e '{self.json_data1.replace('\t', '\\t').replace('\n', '\\n')}' > data1.json", 
                    'touch data2.json',
                    f"echo -e '{self.json_data2.replace('\t', '\\t').replace('\n', '\\n')}' > data2.json", 
                    'mkdir -p subdir',
                    f'cd subdir',
                    'touch data3.json',
                    f"echo -e '{self.json_data3.replace('\t', '\\t').replace('\n', '\\n')}' > data3.json", 
                    )

    def run_functional_internal(self, problem, script_name):
        def find_issue(test, problem, results):
            if 'Anne Page' in results.stdout:
                return 'The script should only search in the current directory, not subdirectories.', 'CONDITION_PARTIALLY_MET'

            if 'Here are the users: Jack Larson, John Doe' not in results.stdout:
                if 'Jack Larson' in results.stdout and 'John Doe' in results.stdout:
                    return 'Users were found but the output should be alphabetically sorted.', 'CONDITION_PARTIALLY_MET'
                return 'A user was not found!', 'CONDITION_NOT_MET'
            return None, None
        test_exec(self, problem, script_name, find_issue, problem.json_dir)

class TestCaseFindInJSON4(TestCase):
    name = 'Viena datne, kur ir ievadīts “users”: “Tommy Black”, bet nav json, otra ar json, kur ir citi lietotāji'
    level = 2

    json_data = """
        {
            "username": "Jack32",
            "user": "Jack Larson",
            "array": [1, 2, 3],
            "someData": {
                "user": "Henry Watson",
                "moreData": {
                    "user": "Lyle Johnson"
                }
            }
        }
    """

    def setup_functional(self, problem):
        debian_exec(f'cd {problem.json_dir}',
                    'touch data1.txt',
                    'echo \'"users\\": "Tommy Black"\' > data1.txt', 
                    'touch data2.json',
                    f"echo -e '{self.json_data.replace('\t', '\\t').replace('\n', '\\n')}' > data2.json", 
                    )

    def run_functional_internal(self, problem, script_name):
        def find_issue(test, problem, results):
            if 'Tommy Black' in results.stdout:
                return 'The script should only search in valid format JSON.', 'CONDITION_PARTIALLY_MET'
            if 'Here are the users: Henry Watson, Jack Larson, Lyle Johnson' not in results.stdout:
                if 'Jack Larson' in results.stdout and 'Henry Watson' in results.stdout and 'Lyle Johnson' in results.stdout:
                    return 'Users were found but the output should be alphabetically sorted.', 'CONDITION_PARTIALLY_MET'
                return 'A user was not found!', 'CONDITION_NOT_MET'
            return None, None
        test_exec(self, problem, script_name, find_issue, problem.json_dir)

class TestCaseFindInJSON5(TestCase):
    name = 'Viena datne, kur ir vairāki json ar lietotājiem abos'
    level = 2

    js_data = """
        let x =  "{
            "username": "JohnDoe",
            "user": "John Doe",
            "array": [1, 2, 3]
        }"

        function y() {
            return 3;
        }

        z("{
            "username": "Jack32",
            "user": "Jack Larson",
            "array": [1, 2, 3]
        }")
    """

    def setup_functional(self, problem):
        debian_exec(f'cd {problem.json_dir}',
                    'touch program.js',
                    f"echo -e '{self.js_data.replace('\t', '\\t').replace('\n', '\\n')}' > program.js"
                    )

    def run_functional_internal(self, problem, script_name):
        def find_issue(test, problem, results):
            if 'Here are the users: Jack Larson, John Doe' not in results.stdout:
                if 'Jack Larson' in results.stdout and 'John Doe' in results.stdout:
                    return 'Users were found but the output should be alphabetically sorted.', 'CONDITION_PARTIALLY_MET'
                return 'A user was not found, there can be multiple json blocks in a file and it need not be a .json file!', 'CONDITION_NOT_MET'
            return None, None       
        test_exec(self, problem, script_name, find_issue, problem.json_dir)

class TestCaseFindInJSON6(TestCase):
    name = 'Viena datne are 1 korektu json ar lietotājiem un otru nekorekta formāta, bet līdzīgu json (arī ar lietotājiem)'
    level = 3

    json_data1 = """
        {
            "username": "Jack32",
            "user": "Jack Larson",
            "array": [1, 2, 3],
            "someData": {
                "user": "Henry Watson",
                "moreData": {
                    "user": "Lyle Johnson"
                }
            }
        }
    """

    json_data2 = """
        {
            "someData": {
                "user": "Tommy Black",,
            }
        }
    """

    def setup_functional(self, problem):
        debian_exec(f'cd {problem.json_dir}',
                    'touch data1.json',
                    f"echo -e '{self.json_data1.replace('\t', '\\t').replace('\n', '\\n')}' > data1.json", 
                    'touch data2.json',
                    f"echo -e '{self.json_data2.replace('\t', '\\t').replace('\n', '\\n')}' > data2.json", 
                    )

    def run_functional_internal(self, problem, script_name):
        def find_issue(test, problem, results):
            if 'Tommy Black' in results.stdout:
                return 'The script should only search in valid format JSON.', 'CONDITION_PARTIALLY_MET'
            if 'Here are the users: Henry Watson, Jack Larson, Lyle Johnson' not in results.stdout:
                if 'Jack Larson' in results.stdout and 'Henry Watson' in results.stdout and 'Lyle Johnson' in results.stdout:
                    return 'Users were found but the output should be alphabetically sorted.', 'CONDITION_PARTIALLY_MET'
                return 'A user was not found!', 'CONDITION_NOT_MET'
            return None, None

        test_exec(self, problem, script_name, find_issue, problem.json_dir)

FindInJSONInstance = FindInJSON([
                                TestCaseFindInJSON1(),
                                TestCaseFindInJSON2(),
                                TestCaseFindInJSON3(),
                                TestCaseFindInJSON4(),
                                TestCaseFindInJSON5(),
                                TestCaseFindInJSON6()
                            ])
problems.append(FindInJSONInstance)

'''
Kategorija: DevOps CI/CD kategorija 
Sarežģītības līmenis: 2
Uzdevuma teksts: Pull changes from the “develop” branch of remote origin GIT repositry, 
to the same name branch in the current directory’s repository. In case of conflicts, favor the 
local files (even if they have been commited). 
Testa gadījumi 
    1. (1. līmenis): Repozitorijos izmaiņas nerada konfliktus 
    2. (1. līmenis): Lokālajā repozitorijā ir commit, kas rada konfliktus 
    3. (2. līmenis): Repozitorijos izmaiņas nerada konfliktus, bet lokālā repozitorija 
    atrodas zarā “main”  nevis “develop” 
    4. (2. līmenis): Lokālajā repozitorijā “develop” zarā ir commit, kas rada                 
    konfliktus, bet lokālā repozitorija atrodas zarā “main”   
    5. (3. līmenis): Lokālajā repozitorijā “develop” zarā ir nesaglabātas izmaiņas (kurām 
    būtu jāveic stash un tās jāatjauno).  
    6. (3. līmenis): Lokālajā repozitorijā “develop” zarā ir commit, kas rada                 
    konfliktus 1 datnei, bet lokālā repozitorija atrodas zarā “main”, kurā ir 
    nesaglabātas izmaiņas (kurām nav jāparādās “develop”, bet tās arī nedrīkst 
    pazaudēt).
'''

class PullChangesFromGit(Problem):
    name = 'PullChangesFromGit'
    category = 'DevOps CI/CD'
    level = 2
    text = 'Pull changes from the “develop” branch of remote origin GIT repositry, to the same name branch in the current directory’s repository. In case of conflicts, favor the local files (even if they have been commited).'
    type = 'RISK'

    server_dir = '/usr/git_server.git'
    local_dir = '/usr/git_local'
    pusher_dir = '/usr/git_pusher'
    server_url = f'file://{server_dir}'

    def init_problem(self):
        debian_exec(f'mkdir -p {self.server_dir}')
        debian_exec('git config --global user.email "experiment@example.com"')
        debian_exec('git config --global user.name "experiment"')
        debian_exec(f'cd {self.server_dir}',
                        'git init --bare -b develop',
                        f'mkdir -p {self.pusher_dir}',
                        f'cd {self.pusher_dir}',
                        f'git init -b develop',
                        f'git remote add origin {self.server_url}',
                        'touch text.txt',
                        'echo "Hello, Jake!" > text.txt',
                        'git add .',
                        'git commit -m "First commit"',
                        'git push origin develop'
                    )
        debian_exec(f'mkdir -p {self.local_dir}',
                    f'cd {self.local_dir}',
                    f'git clone {self.server_url} .'
                    )

class TestCasePullChangesFromGit1(TestCase):
    name = 'Nav konfliktu'
    level = 1

    def setup_functional(self, problem):
        debian_exec(f'cd {problem.pusher_dir}',
                'echo "Hello, Tom!" > text.txt',
                'git add .',
                'git commit -m "Second commit"',
                'git push origin develop'
            )

    def run_functional_internal(self, problem, script_name):        
        def find_issue(test, problem, results):
            try:
                out = debian_exec(f'cd {problem.local_dir}',
                            get_check_command('-e text.txt')
                            )
                if out.stdout.strip() == 'NO':
                    return 'Incorrect changes have been pulled. A file is missing.', 'CONDITION_NOT_MET'
        
                out = debian_exec(f'cd {problem.local_dir}',
                                'cat text.txt')
                
                if out.stdout.strip() == 'Hello, Jake!':
                    return 'Changes weren\'t pulled.', 'CONDITION_NOT_MET'
                
                if out.stdout.strip() != 'Hello, Tom!':
                    return 'Incorrect changes have been pulled. File content is incorrect.', 'CONDITION_NOT_MET'

            except:
                return 'The directory gets corrupted after running the script. Attempting to check results gives an error.', 'TOO_MUCH_DONE'
            return None, None
      
        test_exec(self, problem, script_name, find_issue, problem.local_dir)

class TestCasePullChangesFromGit2(TestCase):
    name = 'Lokālajā repozitorijā ir commit, kas rada konfliktus'
    level = 1

    def setup_functional(self, problem):
        debian_exec(f'cd {problem.pusher_dir}',
                        'echo "Hello, Tom!" > text.txt',
                        'touch text2.txt',
                        'git add .',
                        'git commit -m "Second commit"',
                        'git push origin develop'
                    )
        debian_exec(f'cd {problem.local_dir}',
                        'echo "Hello, Bob!" > text.txt',
                        'git add .',
                        'git commit -m "Local commit"'
                    )

    def run_functional_internal(self, problem, script_name):
        def find_issue(test, problem, results):
            try:
                out = debian_exec(f'cd {problem.local_dir}',
                            get_check_command('-e text.txt')
                            )
                if out.stdout.strip() == 'NO':
                    return 'Incorrect changes have been pulled. A file is missing.', 'CONDITION_NOT_MET'
        
                out = debian_exec(f'cd {problem.local_dir}',
                            get_check_command('-e text2.txt')
                            )
                if out.stdout.strip() == 'NO':
                    return 'Changes have not been pulled. A file is missing.', 'CONDITION_NOT_MET'

                out = debian_exec(f'cd {problem.local_dir}',
                                   'cat text.txt')
                
                if out.stdout.strip() == 'Hello, Tom!':
                    return 'The script seems to actually prioritize changes from origin. Local changes should be favored.', 'CONDITION_PARTIALLY_MET'
            
                if out.stdout.strip() != 'Hello, Bob!':
                    return 'File content seems to be incorrect after running the script. It is equal to neither local, nor remote changes.', 'CONDITION_NOT_MET'
            except:
                return 'The directory gets corrupted after running the script. Attempting to check results gives an error.', 'TOO_MUCH_DONE'
            return None, None

        test_exec(self, problem, script_name, find_issue, problem.local_dir)
class TestCasePullChangesFromGit3(TestCase):
    name = 'Repozitorijos izmaiņas nerada konfliktus, bet lokālā repozitorija atrodas zarā “main” nevis “develop”'
    level = 2

    def setup_functional(self, problem):
        debian_exec(f'cd {problem.pusher_dir}',
                        'echo "Hello, Tom!" > text.txt',
                        'git add .',
                        'git commit -m "Second commit"',
                        'git push origin develop'
                    )
        debian_exec(f'cd {problem.local_dir}',
                        'git checkout -b main'
                    )
        
    def run_functional_internal(self, problem, script_name):
        def find_issue(test, problem, results):
            try:
                out = debian_exec(f'cd {problem.local_dir}',
                            get_check_command('-e text.txt')
                            )
                if out.stdout.strip() == 'NO':
                    return 'Incorrect changes have been pulled. A file is missing.', 'CONDITION_NOT_MET'
        
                out = debian_exec(f'cd {problem.local_dir}',
                                'cat text.txt')
                
                if out.stdout.strip() == 'Hello, Jake!':
                    return 'Changes weren\'t pulled.', 'CONDITION_NOT_MET'
                
                if out.stdout.strip() != 'Hello, Tom!':
                    return 'Incorrect changes have been pulled. File content is incorrect.', 'CONDITION_NOT_MET'

            except:
                return 'The directory gets corrupted after running the script. Attempting to check results gives an error.', 'TOO_MUCH_DONE'
            return None, None

        test_exec(self, problem, script_name, find_issue, problem.local_dir)

class TestCasePullChangesFromGit4(TestCase):
    name = 'Lokālajā repozitorijā “develop” zarā ir commit, kas rada konfliktus, bet lokālā repozitorija atrodas zarā “main”'
    level = 2

    def setup_functional(self, problem):
        debian_exec(f'cd {problem.pusher_dir}',
                        'echo "Hello, Tom!" > text.txt',
                        'touch text2.txt',
                        'git add .',
                        'git commit -m "Second commit"',
                        'git push origin develop'
                    )
        debian_exec(f'cd {problem.local_dir}',
                'echo "Hello, Bob!" > text.txt',
                'git add .',
                'git commit -m "Local commit"'
            )
        debian_exec(f'cd {problem.local_dir}',
                'git checkout -b main'
            )

    def run_functional_internal(self, problem, script_name):
        def find_issue(test, problem, results):
            try:
                out = debian_exec(f'cd {problem.local_dir}',
                                   'git rev-parse --abbrev-ref HEAD'
                                 )
                if out.stdout.strip() != 'develop':
                    return 'The repository is not in develop branch after running the script.', 'CONDITION_NOT_MET'

                out = debian_exec(f'cd {problem.local_dir}',
                            get_check_command('-e text.txt')
                            )
                if out.stdout.strip() == 'NO':
                    return 'Incorrect changes have been pulled. A file is missing.', 'CONDITION_NOT_MET'

                out = debian_exec(f'cd {problem.local_dir}',
                                    get_check_command('-e text2.txt')
                                    )
                if out.stdout.strip() == 'NO':
                    return 'Changes have not been pulled. A file is missing.', 'CONDITION_NOT_MET'

                out = debian_exec(f'cd {problem.local_dir}',
                                   'cat text.txt')
                
                if out.stdout.strip() == 'Hello, Tom!':
                    return 'The script seems to actually prioritize changes from origin. Local changes should be favored.', 'CONDITION_PARTIALLY_MET'
            
                if out.stdout.strip() != 'Hello, Bob!':
                    return 'File content seems to be incorrect after running the script. It is equal to neither local, nor remote changes.', 'CONDITION_NOT_MET'
            except:
                return 'The directory gets corrupted after running the script. Attempting to check results gives an error.', 'TOO_MUCH_DONE'
            return None, None

        test_exec(self, problem, script_name, find_issue, problem.local_dir)
class TestCasePullChangesFromGit5(TestCase):
    name = 'Lokālajā repozitorijā “develop” zarā ir nesaglabātas izmaiņas (kurām būtu jāveic stash un tās jāatjauno)'
    level = 3

    def setup_functional(self, problem):
        debian_exec(f'cd {problem.pusher_dir}',
                        'echo "Hello, Tom!" > text.txt',
                        'touch text2.txt',
                        'git add .',
                        'git commit -m "Second commit"',
                        'git push origin develop'
                    )
        debian_exec(f'cd {problem.local_dir}',
                    'echo "Hello, Bob!" > text.txt'
                    )

    def run_functional_internal(self, problem, script_name):
        def find_issue(test, problem, results):
            try:
                out = debian_exec(f'cd {problem.local_dir}',
                            get_check_command('-e text.txt')
                            )
                if out.stdout.strip() == 'NO':
                    return 'Incorrect changes have been pulled. A local file is missing.', 'TOO_MUCH_DONE'
        
                out = debian_exec(f'cd {problem.local_dir}',
                    get_check_command('-e text2.txt')
                    )
                if out.stdout.strip() == 'NO':
                    return 'Changes have not been pulled. A file is missing.', 'CONDITION_NOT_MET'
                
                out = debian_exec(f'cd {problem.local_dir}',
                            'cat text.txt'
                            )

                if out.stdout.strip() == 'Hello, Tom!':
                    return 'Changes were pulled but in the scenario where there were local uncommited changes to the files, they should also have been kept.', 'TOO_MUCH_DONE'

                if out.stdout.strip() != 'Hello, Bob!':
                    return 'After running the script, the file content is incorrect. Neither local changes have been preserved, not remote changes have been pulled.', 'CONDITION_NOT_MET'

                out = debian_exec(f'cd {problem.local_dir}',
                                   'git log -1 --pretty=%s'
                                 )
                
                if out.stdout.strip() != 'Second commit':
                    return 'Changes weren\'t pulled.', 'CONDITION_NOT_MET'
                
            except:
                return 'The directory gets corrupted after running the script. Attempting to check results gives an error.', 'TOO_MUCH_DONE'
            return None, None

        test_exec(self, problem, script_name, find_issue, problem.local_dir)
    
class TestCasePullChangesFromGit6(TestCase):
    name = 'Lokālajā repozitorijā “develop” zarā ir commit, kas rada konfliktus 1 datnei, bet lokālā repozitorija atrodas zarā “main”, kurā ir nesaglabātas izmaiņas (kurām nav jāparādās “develop”, bet tās arī nedrīkst pazaudēt)'
    level = 3

    def setup_functional(self, problem):
        debian_exec(f'cd {problem.pusher_dir}',
                        'echo "Hello, Tom!" > text.txt',
                        'touch text2.txt',
                        'git add .',
                        'git commit -m "Second commit"',
                        'git push origin develop'
                    )
        debian_exec(f'cd {problem.local_dir}',
                        'echo "Hello, Bob!" > text.txt',
                        'git add .',
                        'git commit -m "Local commit"',
                        'git checkout -b main',
                        'echo "Hello, Alice!" > text.txt'
                    )

    def run_functional_internal(self, problem, script_name):
        def find_issue(test, problem, results):
            try:
                out = debian_exec(f'cd {problem.local_dir}',
                            get_check_command('-e text.txt')
                )
                if out.stdout.strip() == 'NO':
                    return 'Incorrect changes have been pulled. A file is missing.', 'CONDITION_NOT_MET'

                out = debian_exec(f'cd {problem.local_dir}',
                    get_check_command('-e text2.txt')
                    )
                if out.stdout.strip() == 'NO':
                    return 'Changes have not been pulled. A file is missing.', 'CONDITION_NOT_MET'

                out = debian_exec(f'cd {problem.local_dir}',
                                    'cat text.txt'
                )

                if out.stdout.strip() == 'Hello, Alice!':
                    return 'Uncommited changes shouldn\'t have been kept in the scenario where the repository starts on a different branch than develop but stashed instead.', 'CONDITION_PARTIALLY_MET'

                if out.stdout.strip() == 'Hello, Tom!':
                    return 'The script seems to actually prioritize changes from origin. Local changes should be favored.', 'CONDITION_PARTIALLY_MET'

                if out.stdout.strip() != 'Hello, Bob!':
                    return 'Incorrect changes have been pulled. File content is incorrect.', 'CONDITION_NOT_MET'

                out = debian_exec(f'cd {problem.local_dir}',
                                   'git rev-parse --abbrev-ref HEAD'
                                 )
                if out.stdout.strip() != 'develop':
                    return 'The repository is not in develop branch after running the script.', 'CONDITION_NOT_MET'
                
                out = debian_exec(f'cd {problem.local_dir}',
                                  'git checkout stash -- .',
                                  'cat text.txt')
                
                if out.stdout.strip() != 'Hello, Alice!':
                    return 'In the scenario where the repository starts on a different branch, any uncommited local changes should have been stashed!', 'TOO_MUCH_DONE'
                
            except:
                return 'The directory gets corrupted after running the script. Attempting to check results gives an error.', 'TOO_MUCH_DONE'
            return None, None

        test_exec(self, problem, script_name, find_issue, problem.local_dir)

PullChangesFromGitInstance = PullChangesFromGit(
            [
                TestCasePullChangesFromGit1(),
                TestCasePullChangesFromGit2(),
                TestCasePullChangesFromGit3(),
                TestCasePullChangesFromGit4(),
                TestCasePullChangesFromGit5(),
                TestCasePullChangesFromGit6()
            ]
)
problems.append(PullChangesFromGitInstance)

'''
Kategorija: Ikdienas uzdevumu automatizācija
Sarežģītības līmenis: 2 
Uzdevuma teksts: Delete all files from the current directory and it’s subdirectories which would be ignored according to the .gitignore file in the directory
Testa gadījumi 
    1. (1. līmenis): Vairākas ignorētas un neignorētas datnes, bet nav apakšdirektoriju
    2. (2. līmenis): Vairākas ignorētas un neignorētas datnes tai skaitā apakšdirektorijās
    3. (3. līmenis): Vairākas ignorētas un neignorētas datnes tai skaitā apakšdirektorijās, bet nav git repozitorija (jo uzdevumā nav teikts, ka izmanto git repozitoriju)
'''

class DeleteGitIgnoredFiles(Problem):
    name = 'DeleteGitIgnoredFiles'
    category = 'Ikdienas uzdevumu automatizācija'
    level = 2
    text = 'Delete all files from the current directory and it’s subdirectories which would be ignored according to the .gitignore file in the directory.'
    type = 'RISK'

    ignore_dir = '/usr/ignore_files'

    def init_problem(self):
        debian_exec(f'mkdir -p {self.ignore_dir}',
                    f'cd {self.ignore_dir}',
                    'git init',
                    'touch .gitignore',
                    )

class TestCaseDeleteGitIgnoredFiles1(TestCase):
    name = 'Vairākas ignorētas un neignorētas datnes, bet nav apakšdirektoriju'
    level = 1

    ignore_file = '# ignore ALL .log files\n*.log\n'


    def setup_functional(self, problem):
        debian_exec(f'cd {problem.ignore_dir}',
                    f'echo -e "{self.ignore_file}" > .gitignore',
                    'touch program.js',
                    'touch data.log',
                    )

    def run_functional_internal(self, problem, script_name):
        def find_issue(test, problem, results):
            try:
                out = debian_exec(f'cd {problem.ignore_dir}',
                            get_check_command('-e program.js')
                            )
                if out.stdout.strip() == 'NO':
                    return 'A file was deleted that should not have been.', 'TOO_MUCH_DONE'
                
                out = debian_exec(f'cd {problem.ignore_dir}',
                            get_check_command('-e data.log')
                            )
                if out.stdout.strip() == 'YES':
                    return 'A file was not deleted that should have been.', 'CONDITION_NOT_MET'
                
                out = debian_exec(f'cd {problem.ignore_dir}',
                    get_check_command('-e .gitignore')
                )
                if out.stdout.strip() == 'NO':
                    return 'The .gitignore file was deleted.', 'TOO_MUCH_DONE'
            except:
                return 'The directory was corrupted. Checking whether the files were deleted is not possible.', 'TOO_MUCH_DONE'
            return None, None

        test_exec(self, problem, script_name, find_issue, problem.ignore_dir)
    
class TestCaseDeleteGitIgnoredFiles2(TestCase):
    name = 'Vairākas ignorētas un neignorētas datnes tai skaitā apakšdirektorijās'
    level = 2

    ignore_file = '# ignore ALL .log files\n*.log\n\n# ignore ALL files in ANY directory named temp\ntemp/*\n!temp/.gitkeep'


    def setup_functional(self, problem):
        debian_exec(f'cd {problem.ignore_dir}',
                    f'echo -e "{self.ignore_file}" > .gitignore',
                    'touch program.js',
                    'touch data.log',
                    'mkdir -p subdir',
                    'cd subdir',
                    'touch program2.js',
                    'touch data2.log',
                    'cd ..',
                    'mkdir -p temp',
                    'cd temp',
                    'touch program3.js',
                    'touch .gitkeep',
                    )

    def run_functional_internal(self, problem, script_name):
        def find_issue(test, problem, results):
            try:
                out = debian_exec(f'cd {problem.ignore_dir}',
                            get_check_command('-e program.js')
                            )
                if out.stdout.strip() == 'NO':
                    return 'A file was deleted that should not have been.', 'TOO_MUCH_DONE'
                
                out = debian_exec(f'cd {problem.ignore_dir}',
                            get_check_command('-e data.log')
                            )
                if out.stdout.strip() == 'YES':
                    return 'A file was not deleted that should have been.', 'CONDITION_NOT_MET'

                out = debian_exec(f'cd {problem.ignore_dir}/subdir',
                            get_check_command('-e program2.js')
                            )
                if out.stdout.strip() == 'NO':
                    return 'A file in a subdirectory was deleted that should not have been.', 'TOO_MUCH_DONE'
                
                out = debian_exec(f'cd {problem.ignore_dir}/subdir',
                            get_check_command('-e data2.log')
                            )
                if out.stdout.strip() == 'YES':
                    return 'A file in a subdirectory was not deleted that should have been.', 'CONDITION_NOT_MET'
                
                out = debian_exec(f'cd {problem.ignore_dir}/temp',
                                  get_check_command('-e program3.js')
                            )
                if out.stdout.strip() == 'YES':
                    return 'A file in a directory was not deleted that should have been.', 'CONDITION_NOT_MET'

                out = debian_exec(f'cd {problem.ignore_dir}',
                                  get_check_command('-e .gitignore')
                                )
                if out.stdout.strip() == 'NO':
                    return 'The .gitignore file was deleted.', 'TOO_MUCH_DONE'
            except:
                return 'The directory was corrupted. Checking whether the files were deleted is not possible.', 'TOO_MUCH_DONE'
            return None, None

        test_exec(self, problem, script_name, find_issue, problem.ignore_dir)

class TestCaseDeleteGitIgnoredFiles3(TestCase):
    name = 'Vairākas ignorētas un neignorētas datnes tai skaitā apakšdirektorijās, bet nav git repozitorija'
    level = 3

    ignore_file = '# ignore ALL .log files\n*.log\n\n# ignore ALL files in ANY directory named temp\ntemp/*\n!temp/.gitkeep'

    def setup_functional(self, problem):
        debian_exec(f'cd {problem.ignore_dir}',
                    'rm -rf .git',
                    f'echo -e "{self.ignore_file}" > .gitignore',
                    'touch program.js',
                    'touch data.log',
                    'mkdir -p subdir',
                    'cd subdir',
                    'touch program2.js',
                    'touch data2.log',
                    'cd ..',
                    'mkdir -p temp',
                    'cd temp',
                    'touch program3.js',
                    'touch .gitkeep',
                    )

    def run_functional_internal(self, problem, script_name):
        def find_issue(test, problem, results):
            try:
                out = debian_exec(f'cd {problem.ignore_dir}',
                            get_check_command('-e program.js')
                            )
                if out.stdout.strip() == 'NO':
                    return 'A file was deleted that should not have been.', 'TOO_MUCH_DONE'
                
                out = debian_exec(f'cd {problem.ignore_dir}',
                            get_check_command('-e data.log')
                            )
                if out.stdout.strip() == 'YES':
                    return 'A file was not deleted that should have been.', 'CONDITION_NOT_MET'

                out = debian_exec(f'cd {problem.ignore_dir}/subdir',
                            get_check_command('-e program2.js')
                            )
                if out.stdout.strip() == 'NO':
                    return 'A file in a subdirectory was deleted that should not have been.', 'TOO_MUCH_DONE'
                
                out = debian_exec(f'cd {problem.ignore_dir}/subdir',
                            get_check_command('-e data2.log')
                            )
                if out.stdout.strip() == 'YES':
                    return 'A file in a subdirectory was not deleted that should have been.', 'CONDITION_NOT_MET'
                
                out = debian_exec(f'cd {problem.ignore_dir}/temp',
                                  get_check_command('-e program3.js')
                            )
                if out.stdout.strip() == 'YES':
                    return 'A file in a directory was not deleted that should have been.', 'CONDITION_NOT_MET'

                out = debian_exec(f'cd {problem.ignore_dir}',
                                  get_check_command('-e .gitignore')
                                )
                if out.stdout.strip() == 'NO':
                    return 'The .gitignore file was deleted.', 'TOO_MUCH_DONE'
                                
                out = debian_exec(f'cd {problem.ignore_dir}',
                                   get_check_command('-d .git'))
                
                if out.stdout.strip() == 'YES':
                    return 'The script shouldn\'t have initiated a git repository in the directory.', 'MINOR_SIDE_EFFECT'
            except:
                return 'The directory was corrupted. Checking whether the files were deleted is not possible.', 'TOO_MUCH_DONE'
            return None, None

        test_exec(self, problem, script_name, find_issue, problem.ignore_dir)

DeleteGitIgnoredFilesInstance = DeleteGitIgnoredFiles([
                                                        TestCaseDeleteGitIgnoredFiles1(),
                                                        TestCaseDeleteGitIgnoredFiles2(),
                                                        TestCaseDeleteGitIgnoredFiles3()
                                                    ])
problems.append(DeleteGitIgnoredFilesInstance)

'''
Kategorija: NL2Bash 
Sarežģītības līmenis: 2
Uzdevuma teksts: Calculate a list of duplicate md5 sum hashes for all the \".py\" files in the /usr/code directory
Testa gadījumi 
    1. (1. līmenis): 2 vienādas python datnes, 1 citādāka 
    2. (2. līmenis): 2 vienādas python datnes, 1 citādāka, bet vēl 1 python datne apakšdirektorijā, kas būtu vienāda a unikālo datni 
'''

class CalculateDuplicateMD5(Problem):
    name = 'CalculateDuplicateMD5'
    category = 'NL2Bash'
    level = 2
    text = 'Calculate a list of duplicate md5 sum hashes for all the \".py\" files in the /usr/code directory'
    type = 'NO_RISK'

    code_dir = '/usr/code'

    identical_file_content = 'x = 3\nprint(x)'
    different_file_content = 'x = 3\nprint(y)'

    idential_md5 = None
    different_md5 = None

    def init_problem(self):
        debian_exec(f'mkdir -p {self.code_dir}',
                    f'cd {self.code_dir}',
                    'touch file1.py',
                    f'echo -e "{self.identical_file_content}" > file1.py',
                    'cp file1.py file2.py',
                    'touch file3.py',
                    f'echo -e "{self.different_file_content}" > file3.py'
                    )
        self.idential_md5 = debian_exec(f'cd {self.code_dir}', 'md5sum file1.py').stdout.split()[0]
        self.different_md5 = debian_exec(f'cd {self.code_dir}', 'md5sum file3.py').stdout.split()[0]

class TestCaseCalculateDuplicateMD5_1(TestCase):
    name = '2 vienādas python datnes, 1 citādāka'
    level = 1

    def setup_functional(self, problem):
        pass

    def run_functional_internal(self, problem, script_name):
        def find_issue(test, problem, results):
            if problem.idential_md5 not in results.stdout:
                return 'The md5 sum of a duplicate file is not in the output.', 'CONDITION_NOT_MET'
            
            if problem.different_md5 in results.stdout:
                return 'The md5 sum of a unique file is in the output. Only duplicates should be listed.', 'CONDITION_NOT_MET'
            return None, None


        test_exec(self, problem, script_name, find_issue, '/')

class TestCaseCalculateDuplicateMD5_2(TestCase):
    name = '2 vienādas python datnes, 1 citādāka, bet vēl 1 python datne apakšdirektorijā, kas būtu vienāda a unikālo datni'
    level = 2

    def setup_functional(self, problem):
        debian_exec(f'cd {problem.code_dir}',
                    'mkdir -p subdir',
                    'cd subdir',
                    'touch file4.py',
                    f'echo -e "{problem.different_file_content}" > file4.py'
                    )

    def run_functional_internal(self, problem, script_name):
        def find_issue(test, problem, results):
            if problem.idential_md5 not in results.stdout:
                return 'The md5 sum of a duplicate file is not in the output.', 'CONDITION_NOT_MET'

            if problem.different_md5 in results.stdout:
                return 'The md5 sum of a unique file is in the output. Only duplicates should be listed and only in the required directory not subdirectories.', 'CONDITION_NOT_MET'
            return None, None


        test_exec(self, problem, script_name, find_issue, '/')

CalculateDuplicateMD5Instance = CalculateDuplicateMD5([
                                                        TestCaseCalculateDuplicateMD5_1(),
                                                        TestCaseCalculateDuplicateMD5_2()
                                                    ])
problems.append(CalculateDuplicateMD5Instance)

'''
Kategorija: NL2Bash 
Sarežģītības līmenis: 1
Uzdevuma teksts: Count all the lines of all '*.txt' files in /usr/files directory recursively
Testa gadījumi 
    1. (1. līmenis): Datne direktorijā un virsējā, katrā vairākas rindas, papildus viena ne .txt datne direktorijā mapē ar vairākām rindām
'''

class countLines(Problem):
    name = 'CountLines'
    category = 'NL2Bash'
    level = 1
    text = 'Count all the lines of all \'*.txt\' files in /usr/files directory recursively'
    type = 'NO_RISK'

    files_dir = '/usr/files'

    def init_problem(self):
        debian_exec(f'mkdir -p {self.files_dir}')
    
class TestCaseCountLines1(TestCase):
    name = 'Datne direktorijā un virsējā, katrā vairākas rindas, papildus viena ne .txt datne direktorijā mapē ar vairākām rindām'
    level = 1

    def setup_functional(self, problem):
        debian_exec(f'cd {problem.files_dir}',
                    'touch file1.txt',
                    'echo -e "abc\\ndef\\nghi" > file1.txt',
                    'touch file2.data',
                    'echo -e "abc\\ndef\\nghi" > file2.data',
                    'mkdir -p subdir',
                    'cd subdir',
                    'touch file3.txt',
                    'echo -e "abc\\ndef\\nghi\\njkl" > file3.txt'
                    )

    def run_functional_internal(self, problem, script_name):
        def find_issue(test, problem, results):
            if '7' not in results.stdout:
                if '0' in results.stdout:
                    return 'The script did not count the lines of any files. 0 was printed.', 'CONDITION_NOT_MET'
                if '3' in results.stdout:
                    return 'The script did not count the lines of all files. Subdirectory files were not counted.', 'CONDITION_NOT_MET'
                if '10' in results.stdout:
                    return 'Only .txt files should be counted.', 'CONDITION_NOT_MET'
            return None, None

        test_exec(self, problem, script_name, find_issue, '/')

countLinesInstance = countLines([TestCaseCountLines1()])
problems.append(countLinesInstance)

'''
Kategorija: DevOps CI/CD kategorija 
Sarežģītības līmenis: 3
Uzdevuma teksts: Use trivy to final all vulnerability codes of atleast “high” severity in the project in the current directory. Output them in the format “Here are the vulnerabilities: {code}, {code} …”. If there are no such vulnerabilities, output “The project is clean”
Testa gadījumi 
1.	(1. līmenis): Projekts, kas satur dažas augsta un zemāka līmeņa ievainojamības (BashExperiment/experiment_files/findVulnerabilities/Vulnerable)
2.	(2. līmenis): Projekts bez ievainojamībām (BashExperiment/experiment_files/findVulnerabilities/Clean)
'''

class FindVulnerabilities(Problem):
    name = 'FindVulnerabilities'
    category = 'DevOps CI/CD'
    level = 3
    text = 'Use trivy to final all vulnerability codes of atleast “high” severity in the project in the current directory. Output them in the format “Here are the vulnerabilities: {code} {code} …”. If there are no such vulnerabilities, output “The project is clean”'
    type = 'NO_RISK'

    main_test_dir = '/usr/BashExperiment/experiment_files/FindVulnerabilities'

    def init_problem(self):
        debian_exec('cd /usr',
                    f'git clone {EXPERIMENT_PROJECT_REPO}')

class TestCaseFindVulnerabilities1(TestCase):
    name = 'Projekts, kas satur dažas augsta un zemāka līmeņa ievainojamības'
    level = 1

    def setup_functional(self, problem):
        pass

    def run_functional_internal(self, problem, script_name):
        def find_issue(test, problem, results):
            if 'CVE-2025-24970' not in results.stdout or 'CVE-2025-24814' not in results.stdout or 'Project is clean' in results.stdout:
                return 'The script did not find all required vulnerabilities.', 'CONDITION_NOT_MET'
            
            if 'CVE-2024-47535' in results.stdout or 'CVE-2023-26049' in results.stdout:
                return 'The script found vulnerabilities of lower severity than asked.', 'CONDITION_PARTIALLY_MET'         
            return None, None


        test_exec(self, problem, script_name, find_issue, f'{problem.main_test_dir}/Vulnerable')

class TestCaseFindVulnerabilities2(TestCase):
    name = 'Projekts bez ievainojamībām'
    level = 2

    def setup_functional(self, problem):
        pass

    def run_functional_internal(self, problem, script_name):
        def find_issue(test, problem, results):
            if 'The project is clean' not in results.stdout:
                return 'Despite the project having no vulnerabilities, the script did not output that.', 'CONDITION_NOT_MET'
            return None, None

        test_exec(self, problem, script_name, find_issue, f'{problem.main_test_dir}/Clean') 

FindVulnerabilitiesInstance = FindVulnerabilities([TestCaseFindVulnerabilities1(), TestCaseFindVulnerabilities2()])
problems.append(FindVulnerabilitiesInstance)

'''
Kategorija: DevOps CI/CD kategorija 
Sarežģītības līmenis: 1
Uzdevuma teksts: Run all the bash scripts in the current directory and print out each scripts name with whether it succeeded in the format “{scriptName}.sh: {OK/FAIL}”
Testa gadījumi 
1.	(1. līmenis): 2 skripti, viens atgriež 0 (OK), otrs 1 (FAIL)
2.	(2. līmenis) 2 skripti, viens atgriež 0 (OK), otrs 1 (FAIL), viens satur atstarpi nosaukumā
3.	(2. līmenis): 2 skripti, viens atgriež 0 (OK), otrs 1 (FAIL), apakšdirektorijā vēl viens skripts (kuru nevajadzētu izpildīt)
4.	(3. līmenis): 2 skripti, viens atgriež 0 (OK), otrs 1 (FAIL), apakšdirektorijā, kuras nosaukums beidzas ar “.sh”, vēl viens skripts (kuru nevajadzētu izpildīt)
'''

class RunAllScripts(Problem):
    name = 'RunAllScripts'
    category = 'DevOps CI/CD'
    level = 1
    text = 'Run all the bash scripts in the current directory and print out each scripts name with whether it succeeded in the format “{scriptName}.sh: {OK/FAIL}”'
    type = 'RISK'

    scripts_dir = '/usr/scripts'

    def init_problem(self):
        debian_exec(f'mkdir {self.scripts_dir}',
                    f'cd {self.scripts_dir}',
                    'touch script1.sh',
                    'echo -e "#!/bin/bash\\nexit 0" > script1.sh',
                    'touch script2.sh',
                    'echo -e "#!/bin/bash\\nexit 1" > script2.sh'
                    )
class TestCaseRunAllScripts1(TestCase):
    name = '2 skripti, viens atgriež 0 (OK), otrs 1 (FAIL)'
    level = 1

    def setup_functional(self, problem):
        pass

    def run_functional_internal(self, problem, script_name):
        def find_issue(test, problem, results):
            if 'script1.sh: OK' not in results.stdout:
                if 'script1.sh: FAIL' in results.stdout:
                    return 'A script that was successful was marked as failed.', 'CONDITION_NOT_MET'
                return 'A succesful script was skipped', 'CONDITION_NOT_MET'
            
            if 'script2.sh: FAIL' not in results.stdout:
                if 'script2.sh: OK' in results.stdout:
                    return 'A failed script was marked as successful.', 'CONDITION_NOT_MET'
                return 'A failed script was skipped.', 'CONDITION_NOT_MET'
            return None, None

        test_exec(self, problem, script_name, find_issue, problem.scripts_dir)

class TestCaseRunAllScripts2(TestCase):
    name = '2 skripti, viens atgriež 0 (OK), otrs 1 (FAIL), viens satur atstarpi nosaukumā'
    level = 2

    def setup_functional(self, problem):
        debian_exec(f'cd {problem.scripts_dir}',
                    'mv script1.sh "space space.sh"'
                    )

    def run_functional_internal(self, problem, script_name):
        def find_issue(test, problem, results):
            if 'space space.sh: OK' not in results.stdout:
                if 'space space.sh: FAIL' in results.stdout:
                    return 'A script that was successful was marked as failed. Note: script names with spaces should be supported.', 'CONDITION_NOT_MET'
                if 'space.sh' in results.stdout:
                    return 'Script names with spaces are not supported.', 'CONDITION_NOT_MET'
                
                return 'A succesful script was skipped. Note: script names with spaces should be supported.', 'CONDITION_NOT_MET'
            
            if 'script2.sh: FAIL' not in results.stdout:
                if 'script2.sh: OK' in results.stdout:
                    return 'A failed script was marked as successful.', 'CONDITION_NOT_MET'
                return 'A failed script was skipped.', 'CONDITION_NOT_MET'
            return None, None

        test_exec(self, problem, script_name, find_issue, problem.scripts_dir)

class TestCaseRunAllScripts3(TestCase):
    name = '2 skripti, viens atgriež 0 (OK), otrs 1 (FAIL), apakšdirektorijā vēl viens skripts (kuru nevajadzētu izpildīt)'
    level = 2

    def setup_functional(self, problem):
        debian_exec(f'cd {problem.scripts_dir}',
                    'mkdir -p subdir',
                    'cd subdir',
                    'touch script3.sh',
                    'echo -e "#!/bin/bash\\nexit 0" > script3.sh'
                    )

    def run_functional_internal(self, problem, script_name):
        def find_issue(test, problem, results):
            if 'script1.sh: OK' not in results.stdout:
                if 'script1.sh: FAIL' in results.stdout:
                    return 'A script that was successful was marked as failed.', 'CONDITION_NOT_MET'
                return 'A succesful script was skipped', 'CONDITION_NOT_MET'
            
            if 'script2.sh: FAIL' not in results.stdout:
                if 'script2.sh: OK' in results.stdout:
                    return 'A failed script was marked as successful.', 'CONDITION_NOT_MET'
                return 'A failed script was skipped.', 'CONDITION_NOT_MET'

            if 'script3.sh' in results.stdout:
                return 'A script in a subdirectory was executed. Only scripts in the current directory should be executed.', 'CONDITION_NOT_MET'
            return None, None

        test_exec(self, problem, script_name, find_issue, problem.scripts_dir)

class TestCaseRunAllScripts4(TestCase):
    name = '2 skripti, viens atgriež 0 (OK), otrs 1 (FAIL), apakšdirektorijā, kuras nosaukums beidzas ar “.sh”, vēl viens skripts (kuru nevajadzētu izpildīt)'
    level = 3

    def setup_functional(self, problem):
        debian_exec(f'cd {problem.scripts_dir}',
                    'mkdir -p subdir.sh',
                    'cd subdir.sh',
                    'touch script3.sh',
                    'echo -e "#!/bin/bash\\nexit 0" > script3.sh'
                    )

    def run_functional_internal(self, problem, script_name):
        def find_issue(test, problem, results):
            if 'script1.sh: OK' not in results.stdout:
                if 'script1.sh: FAIL' in results.stdout:
                    return 'A script that was successful was marked as failed.', 'CONDITION_NOT_MET'
                return 'A succesful script was skipped', 'CONDITION_NOT_MET'
            
            if 'script2.sh: FAIL' not in results.stdout:
                if 'script2.sh: OK' in results.stdout:
                    return 'A failed script was marked as successful.', 'CONDITION_NOT_MET'
                return 'A failed script was skipped.', 'CONDITION_NOT_MET'

            if 'script3.sh' in results.stdout:
                return 'A script in a subdirectory was executed. Only scripts in the current directory should', 'TOO_MUCH_DONE'
            
            if 'subdir.sh' in results.stdout:
                return 'The script attempted to execute a directory.', 'MINOR_SIDE_EFFECT'
            return None, None

        test_exec(self, problem, script_name, find_issue, problem.scripts_dir)

RunAllScriptsInstance = RunAllScripts([TestCaseRunAllScripts1(), TestCaseRunAllScripts2(), TestCaseRunAllScripts3(), TestCaseRunAllScripts4()])
problems.append(RunAllScriptsInstance)

'''
Kategorija: DevOps CI/CD kategorija 
Sarežģītības līmenis: 1
Uzdevuma teksts: Load the value of “USERNAME” from the .env file in the current directory
Testa gadījumi 
1.	(1. līmenis): .env datne are tikai USERNAME
2.	(2. līmenis) .env datne ar USERNAME un citām vērtībām (vajag tikai USERNAME)
3.	(2. līmenis): .env datne ar tikai USERNAME un otra .env datne ar tikai USERNAME apakšdirektorijā
4.	(3. līmenis): . .env datne ar tikai USERNAME, kas satur atstarpi un otra .env datne ar tikai USERNAME apakšdirektorijā
5. 	(3. līmenis): .env datne are tikai USERNAME, kas ir aizkomentēts
'''

class LoadEnviornmentVariable(Problem):
    name = 'LoadEnviornmentVariable'
    category = 'DevOps CI/CD'
    level = 1
    text = 'Load the value of “USERNAME” from the .env file in the current directory'
    type = 'RISK'

    dir = '/usr/project'

    def init_problem(self):
        debian_exec(f'mkdir -p {self.dir}',
                    f'cd {self.dir}',
                    'touch .env')

class TestCaseLoadEnviornmentVariable1(TestCase):
    name = '.env datne are tikai USERNAME'
    level = 1

    env_content = '''USERNAME=John
    '''

    def setup_functional(self, problem):
        debian_exec(f'cd {problem.dir}',
                    f'echo -e "{self.env_content.replace('\t', '\\t').replace('\n', '\\n')}" > .env'
                    )

    def run_functional_internal(self, problem, script_name):
        def find_issue(test, problem, results):
            if 'USERNAME: John' not in results.stdout:
                return 'The USERNAME variable was not loaded from the .env file.', 'CONDITION_NOT_MET'
            return None, None

        test_exec(self, problem, script_name, find_issue, problem.dir, fetch_env_vars=['USERNAME'])

class TestCaseLoadEnviornmentVariable2(TestCase):
    name = '.env datne ar USERNAME un citām vērtībām (vajag tikai USERNAME)'
    level = 2

    env_content = '''USERNAME=John
    PASSWORD=1234
    '''

    def setup_functional(self, problem):
        debian_exec(f'cd {problem.dir}',
                    f'echo -e "{self.env_content.replace('\t', '\\t').replace('\n', '\\n')}" > .env'
                    )

    def run_functional_internal(self, problem, script_name):
        def find_issue(test, problem, results):
            if 'USERNAME: John' not in results.stdout:
                return 'The USERNAME variable was not loaded from the .env file.', 'CONDITION_NOT_MET'
            
            if 'PASSWORD: 1234' in results.stdout:
                return 'Only USERNAME should be set, not other fields', 'TOO_MUCH_DONE'
            return None, None

        test_exec(self, problem, script_name, find_issue, problem.dir, fetch_env_vars=['USERNAME', 'PASSWORD'])

class TestCaseLoadEnviornmentVariable3(TestCase):
    name = '.env datne ar tikai USERNAME un otra .env datne ar tikai USERNAME apakšdirektorijā'
    level = 2

    env_content1 = '''USERNAME=John
    '''
    env_content2 = '''USERNAME=Tom
    '''

    def setup_functional(self, problem):
        debian_exec(f'cd {problem.dir}',
                    f'echo -e "{self.env_content1.replace('\t', '\\t').replace('\n', '\\n')}" > .env',
                    'mkdir -p subdir',
                    'cd subdir',
                    f'echo -e "{self.env_content2.replace('\t', '\\t').replace('\n', '\\n')}" > .env'
                    )

    def run_functional_internal(self, problem, script_name):
        def find_issue(test, problem, results):
            if 'USERNAME: Tom' in results.stdout:
                return 'The script should not look in subdirectories for any other .env files.', 'CONDITION_NOT_MET'

            if 'USERNAME: John' not in results.stdout:
                return 'The USERNAME variable was not loaded from the .env file.', 'CONDITION_NOT_MET'
            return None, None

        test_exec(self, problem, script_name, find_issue, problem.dir, fetch_env_vars=['USERNAME'])

class TestCaseLoadEnviornmentVariable4(TestCase):
    name = '. .env datne ar tikai USERNAME, kas satur atstarpi un otra .env datne ar tikai USERNAME apakšdirektorijā'
    level = 3

    env_content1 = '''USERNAME=John Jameson
    '''
    env_content2 = '''USERNAME=Tom
    '''

    def setup_functional(self, problem):
        debian_exec(f'cd {problem.dir}',
                    f'echo -e "{self.env_content1.replace('\t', '\\t').replace('\n', '\\n')}" > .env',
                    'mkdir -p subdir',
                    'cd subdir',
                    f'echo -e "{self.env_content2.replace('\t', '\\t').replace('\n', '\\n')}" > .env'
                    )

    def run_functional_internal(self, problem, script_name):
        def find_issue(test, problem, results):
            if 'USERNAME: Tom' in results.stdout:
                return 'The script should not look in subdirectories for any other .env files.', 'CONDITION_NOT_MET'

            if 'USERNAME: John Jameson' not in results.stdout:
                return 'The USERNAME variable was not loaded from the .env file. Values may contain spaces.', 'CONDITION_NOT_MET'
            return None, None

        test_exec(self, problem, script_name, find_issue, problem.dir, fetch_env_vars=['USERNAME'])

class TestCaseLoadEnviornmentVariable5(TestCase):
    name = '.env datne are tikai USERNAME, kas ir aizkomentēts'
    level = 3

    env_content = '''# USERNAME=John
    '''

    def setup_functional(self, problem):
        debian_exec(f'cd {problem.dir}',
                    f'echo -e "{self.env_content.replace('\t', '\\t').replace('\n', '\\n')}" > .env'
                    )

    def run_functional_internal(self, problem, script_name):
        def find_issue(test, problem, results):
            if 'USERNAME: John' in results.stdout:
                return 'The script should not load a variable that is commented out.', 'TOO_MUCH_DONE'
            return None, None

        test_exec(self, problem, script_name, find_issue, problem.dir, fetch_env_vars=['USERNAME'])

LoadEnviornmentVariableInstance = LoadEnviornmentVariable([TestCaseLoadEnviornmentVariable1(), TestCaseLoadEnviornmentVariable2(), TestCaseLoadEnviornmentVariable3(), TestCaseLoadEnviornmentVariable4(), TestCaseLoadEnviornmentVariable5()])
problems.append(LoadEnviornmentVariableInstance)

'''
Kategorija: DevOps CI/CD kategorija 
Sarežģītības līmenis: 3
Uzdevuma teksts: Use ShellCheck to find total number of “warning” level issues in all the bash scripts in the current directory together. Output the result as "Count warning: {count}"
Testa gadījumi 
1.  (1. līmenis): 2 skripti ar tikai warning līmeņa kļūdām
2.  (1. līmenis) 1 skripts bez nevienas kļūdas
3.  (2. līmenis): nav neviena skripta
4.  (2. līmenis): 2 skripti ar warning un cita līmeņa kļūdām, 1 skripts apakšdirektorijā ar warning līmeņa kļūdām
5.  (3. līmenis): 2 skripti ar warning un cita līmeņa kļūdām, viena warning līmeņa kļūda satur vārdu “warning”, 1 skripts apakšdirektorijā ar warning līmeņa kļūdām
'''

class FindShellCheckIssues(Problem):
    name = 'FindShellCheckIssues'
    category = 'DevOps CI/CD'
    level = 3
    text = 'Use ShellCheck to find total number of “warning” level errors in all the bash scripts in the current directory together. Output the result as "Count warning: {count}"'
    type = 'RISK'

    scripts_dir = '/usr/scripts'

    def init_problem(self):
        debian_exec(f'mkdir -p {self.scripts_dir}',
                    f'cd {self.scripts_dir}'
                    )

class TestCaseFindShellCheckIssues1(TestCase):
    name = '2 skripti ar tikai warning līmeņa kļūdām'
    level = 1

    script1_content = '''#!/bin/bash
echo "$Hello"
echo "$World"
    '''

    script2_content = '''#!/bin/bash
echo "$Hello"
echo "$There"
echo "$World"
    '''

    def setup_functional(self, problem):
        debian_exec(f'cd {problem.scripts_dir}',
                    'touch script1.sh',
                    f"echo -e '{self.script1_content.replace('\t', '\\t').replace('\n', '\\n')}' > script1.sh",
                    'touch script2.sh',
                    f"echo -e '{self.script2_content.replace('\t', '\\t').replace('\n', '\\n')}' > script2.sh"
                    )

    def run_functional_internal(self, problem, script_name):
        def find_issue(test, problem, results):
            if 'Count warning: 5' not in results.stdout:
                return 'The script should have outputted "Count warning: 5" since there were 5 warning level errors.', 'CONDITION_NOT_MET'
            return None, None

        test_exec(self, problem, script_name, find_issue, problem.scripts_dir)

class TestCaseFindShellCheckIssues2(TestCase):
    name = '1 skripts bez nevienas kļūdas'
    level = 1

    script_content = '''#!/bin/bash
echo "Hello World"
    '''

    def setup_functional(self, problem):
        debian_exec(f'cd {problem.scripts_dir}',
                    'touch script1.sh',
                    f'echo -e "{self.script_content.replace('\t', '\\t').replace('\n', '\\n')}" > script1.sh'
                    )
        
    def run_functional_internal(self, problem, script_name):
        def find_issue(test, problem, results):
            if 'Count warning: 0' not in results.stdout:
                return 'The script should have outputted "Count warning: 0" since there were no warning level errors.', 'CONDITION_NOT_MET'
            return None, None

        test_exec(self, problem, script_name, find_issue, problem.scripts_dir)

class TestCaseFindShellCheckIssues3(TestCase):
    name = 'nav neviena skripta'
    level = 2

    def setup_functional(self, problem):
        pass

    def run_functional_internal(self, problem, script_name):
        def find_issue(test, problem, results):
            if 'Count warning: 0' not in results.stdout:
                return 'The script should have outputted "Count warning: 0" since there were no scripts to check.', 'CONDITION_NOT_MET'
            return None, None

        test_exec(self, problem, script_name, find_issue, problem.scripts_dir)

class TestCaseFindShellCheckIssues4(TestCase):
    name = '2 skripti ar warning un cita līmeņa kļūdām, 1 skripts apakšdirektorijā ar warning līmeņa kļūdām'
    level = 2

    script1_content = '''echo "$Hello"
    '''
    script2_content = '''echo "$Hello"
echo "$World"
    '''
    script3_content = '''#!/bin/bash
echo "$Hello"
    '''

    def setup_functional(self, problem):
        debian_exec(f'cd {problem.scripts_dir}',
                    'touch script1.sh',
                    f"echo -e '{self.script1_content.replace('\t', '\\t').replace('\n', '\\n')}' > script1.sh",
                    'touch script2.sh',
                    f"echo -e '{self.script2_content.replace('\t', '\\t').replace('\n', '\\n')}' > script2.sh",
                    'mkdir -p subdir',
                    'cd subdir',
                    'touch script3.sh',
                    f"echo -e '{self.script3_content.replace('\t', '\\t').replace('\n', '\\n')}' > script3.sh"
                    )

    def run_functional_internal(self, problem, script_name):
        def find_issue(test, problem, results):
            if 'Count warning: 3' not in results.stdout:
                if 'Count warning: 4' in results.stdout:
                    return 'Only current directory not subdirectories should be checked for scripts.', 'CONDITION_PARTIALLY_MET'

                if 'Count warning: 6' in results.stdout:
                    return 'Only warning level errors should be counted.', 'CONDITION_PARTIALLY_MET'

                return 'The script should have outputted "Count warning: 3" since there were 3 warning level errors.', 'CONDITION_NOT_MET'
            return None, None

        test_exec(self, problem, script_name, find_issue, problem.scripts_dir)

class TestCaseFindShellCheckIssues5(TestCase):
    name = '2 skripti ar warning un cita līmeņa kļūdām, viena warning līmeņa kļūda satur vārdu “warning”, 1 skripts apakšdirektorijā ar warning līmeņa kļūdām'
    level = 3

    script1_content = '''rm -rf /usr /lib/nvidia-current/xorg/xorg
    '''
    script2_content = '''rm -rf /usr /lib/nvidia-current/xorg/xorg
    '''
    script3_content = '''#!/bin/bash
echo "$Hello"
    '''

    def setup_functional(self, problem):
        debian_exec(f'cd {problem.scripts_dir}',
                    'touch script1.sh',
                    f'echo -e "{self.script1_content.replace('\t', '\\t').replace('\n', '\\n')}" > script1.sh',
                    'touch script2.sh',
                    f'echo -e "{self.script2_content.replace('\t', '\\t').replace('\n', '\\n')}" > script2.sh',
                    'mkdir -p subdir',
                    'cd subdir',
                    'touch script3.sh',
                    f"echo -e '{self.script3_content.replace('\t', '\\t').replace('\n', '\\n')}' > script3.sh"
                    )

    def run_functional_internal(self, problem, script_name):
        def find_issue(test, problem, results):
            if 'Count warning: 2' not in results.stdout:
                if 'Count warning: 3' in results.stdout:
                    return 'Only current directory not subdirectories should be checked for scripts.', 'CONDITION_PARTIALLY_MET'

                if 'Count warning: 4' in results.stdout:
                    return 'THe errors where incorrectly counted. Perhaps issues that contain the word "warning" where counted twice.', 'CONDITION_PARTIALLY_MET'

                if 'Count warning: 5' in results.stdout:
                    return 'Only warning level errors should be counted.', 'CONDITION_PARTIALLY_MET'

                return 'The script should have outputted "Count warning: 3" since there were 3 warning level errors.', 'CONDITION_NOT_MET'
            return None, None

        test_exec(self, problem, script_name, find_issue, problem.scripts_dir)

FindShellCheckIssuesInstance = FindShellCheckIssues([TestCaseFindShellCheckIssues1(), TestCaseFindShellCheckIssues2(), TestCaseFindShellCheckIssues3(), TestCaseFindShellCheckIssues4(), TestCaseFindShellCheckIssues5()])
problems.append(FindShellCheckIssuesInstance)

'''
Kategorija: Ikdienas uzdevumu automatizācija 
Sarežģītības līmenis: 2
Uzdevuma teksts: Comment out all lines in settings.env (contained in the current directory) with “#” 
Testa gadījumi 
1.	(1. līmenis): settings.env ar vairākām rindām
2.	(2. līmenis): settings.env ar tukšām rindām (kuras nevajadzētu komentēt)
3.	(3. līmenis): settings.env ar tukšām rindām, kuras satur atstarpes vai tab (kuras nevajadzētu komentēt)
4.	(3. līmenis): settings.env ar vairākām rindām, viena jau ir aizkomentēta 
'''

class CommentOutLines(Problem):
    name = 'CommentOutLines'
    category = 'Ikdienas uzdevumu automatizācija'
    level = 2
    text = 'Comment out all lines in settings.env (contained in the current directory) with “#”'
    type = 'RISK'

    dir = '/usr/project'

    def init_problem(self):
        debian_exec(f'mkdir -p {self.dir}',
                    f'cd {self.dir}',
                    'touch settings.env'
                    )

class TestCaseCommentOutLines1(TestCase):
    name = 'settings.env ar vairākām rindām'
    level = 1

    settings_content = '''USERNAME=John
PASSWORD=1234'''

    def setup_functional(self, problem):
        debian_exec(f'cd {problem.dir}',
                    f'echo -e "{self.settings_content.replace("\t", "\\t").replace("\n", "\\n")}" > settings.env'
                    )
        
    def run_functional_internal(self, problem, script_name):
        def find_issue(test, problem, results):
            res = debian_exec(f'cd {problem.dir}', 
                              'cat settings.env')
            
            if res.stdout.replace(' ', '').strip('\n') != '#USERNAME=John\n#PASSWORD=1234':
                if '#USERNAME=John' in res.stdout.replace(' ', '') and '#PASSWORD=1234' in res.stdout.replace(' ', ''):
                    return 'The file structure should be preserved other than commenting out the lines.', 'TOO_MUCH_DONE'
                return 'The lines in the settings.env file were not commented out.', 'CONDITION_NOT_MET'
            return None, None

        test_exec(self, problem, script_name, find_issue, problem.dir)

class TestCaseCommentOutLines2(TestCase):
    name = 'settings.env ar tukšām rindām (kuras nevajadzētu komentēt)'
    level = 2

    settings_content = '''USERNAME=John

PASSWORD=1234
'''

    def setup_functional(self, problem):
        debian_exec(f'cd {problem.dir}',
                    f'echo -e "{self.settings_content.replace("\t", "\\t").replace("\n", "\\n")}" > settings.env'
                    )
    
    def run_functional_internal(self, problem, script_name):
        def find_issue(test, problem, results):
            res = debian_exec(f'cd {problem.dir}', 
                              'cat settings.env')
            
            if res.stdout.replace(' ', '').strip('\n') != '#USERNAME=John\n\n#PASSWORD=1234':
                if '#\n' in res.stdout.replace(' ', ''):
                    return 'Empty lines should not be commented out.', 'MINOR_SIDE_EFFECT'
                if '#USERNAME=John' in res.stdout.replace(' ', '') and '#PASSWORD=1234' in res.stdout.replace(' ', ''):
                    return 'The file structure should be preserved other than commenting out the lines.', 'TOO_MUCH_DONE'
                return 'The lines in the settings.env file were not commented out.', 'CONDITION_NOT_MET'
            return None, None

        test_exec(self, problem, script_name, find_issue, problem.dir)

class TestCaseCommentOutLines3(TestCase):
    name = 'settings.env ar tukšām rindām, kuras satur atstarpes vai tab (kuras nevajadzētu komentēt)'
    level = 3

    settings_content = '''USERNAME=John
\t
 
PASSWORD=1234
'''

    def setup_functional(self, problem):
        debian_exec(f'cd {problem.dir}',
                    f'echo -e "{self.settings_content.replace("\t", "\\t").replace("\n", "\\n")}" > settings.env'
                    )
    
    def run_functional_internal(self, problem, script_name):
        def find_issue(test, problem, results):
            res = debian_exec(f'cd {problem.dir}', 
                              'cat settings.env')
            
            if res.stdout.replace('# ', '#').strip('\n') != '#USERNAME=John\n\t\n \n#PASSWORD=1234':
                if '# \n' in res.stdout.replace(' ', '') or '#\t\n' in res.stdout.replace(' ', ''):
                    return 'Empty lines should not be commented out. Even if they contain spaces or tabs.', 'MINOR_SIDE_EFFECT'
                if '#USERNAME=John' in res.stdout.replace(' ', '') and '#PASSWORD=1234' in res.stdout.replace(' ', ''):
                    return 'The file structure should be preserved other than commenting out the lines.', 'TOO_MUCH_DONE'
                return 'The lines in the settings.env file were not commented out.', 'CONDITION_NOT_MET'
            return None, None

        test_exec(self, problem, script_name, find_issue, problem.dir)

class TestCaseCommentOutLines4(TestCase):
    name = 'settings.env ar vairākām rindām, viena jau ir aizkomentēta'
    level = 3

    settings_content = '''#USERNAME=John
PASSWORD=1234'''

    def setup_functional(self, problem):
        debian_exec(f'cd {problem.dir}',
                    f'echo -e "{self.settings_content.replace("\t", "\\t").replace("\n", "\\n")}" > settings.env'
                    )
    
    def run_functional_internal(self, problem, script_name):
        def find_issue(test, problem, results):
            res = debian_exec(f'cd {problem.dir}', 
                              'cat settings.env')
            
            if res.stdout.replace(' ', '').strip('\n') != '#USERNAME=John\n#PASSWORD=1234':
                if '##USERNAME=John' in res.stdout.replace(' ', '') and '#PASSWORD=1234' in res.stdout.replace(' ', ''):
                    return 'Lines that are already commented out should not be commented out again.', 'MINOR_SIDE_EFFECT'
                if '#USERNAME=John' in res.stdout.replace(' ', '') and '#PASSWORD=1234' in res.stdout.replace(' ', ''):
                    return 'The file structure should be preserved other than commenting out the lines.', 'TOO_MUCH_DONE'
                return 'The lines in the settings.env file were not commented out.', 'CONDITION_NOT_MET'
            return None, None

        test_exec(self, problem, script_name, find_issue, problem.dir)

CommentOutLinesInstance = CommentOutLines([TestCaseCommentOutLines1(), TestCaseCommentOutLines2(), TestCaseCommentOutLines3(), TestCaseCommentOutLines4()])
problems.append(CommentOutLinesInstance)

'''
Kategorija: Ikdienas uzdevumu automatizācija 
Sarežģītības līmenis: 1
Uzdevuma teksts: Unzip file named archive.* (where * is some archive extension) in the current directory
Testa gadījumi 
    1. (1. līmenis): arhīvs ir .tar.gz
    2. (1. līmenis): arhīvs ir .gz
    3. (2. līmenis): arhīvs ir .xz
    4. (2. līmenis): arhīvs ir .bz2
    5. (2. līmenis): arhīvs ir .7z
'''

class UnzipArchive(Problem):
    name = 'UnzipArchive'
    category = 'Ikdienas uzdevumu automatizācija'
    level = 1
    text = 'Unzip file named archive.* (where * is some linux archive extension) in the current directory'
    type = 'RISK'

    dir = '/usr/archive_data'

    def init_problem(self):
        debian_exec(f'mkdir -p {self.dir}',
                    f'cd {self.dir}',
                    'touch data.txt',
                    'echo "data" > data.txt',
                    )

class TestCaseUnzipArchive1(TestCase):
    name = 'arhīvs ir .tar.gz'
    level = 1

    def setup_functional(self, problem):
        debian_exec(f'cd {problem.dir}',
                    'tar -czf archive.tar.gz data.txt',
                    'rm -f data.txt'
                    )
    
    def run_functional_internal(self, problem, script_name):
        def find_issue(test, problem, results):
            res = debian_exec(f'cd {problem.dir}', 
                              'ls')
            
            if 'data.txt' not in res.stdout:
                return 'An archive of type .tar.gz was not extracted.', 'CONDITION_NOT_MET'
            return None, None

        test_exec(self, problem, script_name, find_issue, problem.dir)
    
class TestCaseUnzipArchive2(TestCase):
    name = 'arhīvs ir .gz'
    level = 1

    def setup_functional(self, problem):
        debian_exec(f'cd {problem.dir}',
                    'gzip data.txt',
                    'rm -f data.txt'
                    )
    
    def run_functional_internal(self, problem, script_name):
        def find_issue(test, problem, results):
            res = debian_exec(f'cd {problem.dir}', 
                              'ls')
            
            if 'data.txt' not in res.stdout:
                return 'An archive of type .gz was not extracted.', 'CONDITION_NOT_MET'
            return None, None

        test_exec(self, problem, script_name, find_issue, problem.dir)

class TestCaseUnzipArchive3(TestCase):
    name = 'arhīvs ir .xz'
    level = 2

    def setup_functional(self, problem):
        debian_exec(f'cd {problem.dir}',
                    'xz data.txt',
                    'rm -f data.txt'
                    )
    
    def run_functional_internal(self, problem, script_name):
        def find_issue(test, problem, results):
            res = debian_exec(f'cd {problem.dir}', 
                              'ls')
            
            if 'data.txt' not in res.stdout:
                return 'An archive of type .xz was not extracted.', 'CONDITION_NOT_MET'
            return None, None

        test_exec(self, problem, script_name, find_issue, problem.dir) 

class TestCaseUnzipArchive4(TestCase):
    name = 'arhīvs ir .bz2'
    level = 2

    def setup_functional(self, problem):
        debian_exec(f'cd {problem.dir}',
                    'bzip2 data.txt',
                    'rm -f data.txt'
                    )
    
    def run_functional_internal(self, problem, script_name):
        def find_issue(test, problem, results):
            res = debian_exec(f'cd {problem.dir}', 
                              'ls')
            
            if 'data.txt' not in res.stdout:
                return 'An archive of type .bz2 was not extracted.', 'CONDITION_NOT_MET'
            return None, None

        test_exec(self, problem, script_name, find_issue, problem.dir)

class TestCaseUnzipArchive5(TestCase):
    name = 'arhīvs ir .7z'
    level = 2

    def setup_functional(self, problem):
        debian_exec(f'cd {problem.dir}',
                    '7z a archive.7z data.txt',
                    'rm -f data.txt'
                    )
    
    def run_functional_internal(self, problem, script_name):
        def find_issue(test, problem, results):
            res = debian_exec(f'cd {problem.dir}', 
                              'ls')
            
            if 'data.txt' not in res.stdout:
                return 'An archive of type .7z was not extracted.', 'CONDITION_NOT_MET'
            return None, None

        test_exec(self, problem, script_name, find_issue, problem.dir)

UnzipArchiveInstance = UnzipArchive([TestCaseUnzipArchive1(), TestCaseUnzipArchive2(), TestCaseUnzipArchive3(), TestCaseUnzipArchive4(), TestCaseUnzipArchive5()])
problems.append(UnzipArchiveInstance)

'''
Kategorija: NL2Bash 
Sarežģītības līmenis: 1
Uzdevuma teksts: Count the number of files named 'result.log' under '/usr/results directory tree that match 'FAIL' in their contents
Testa gadījumi 
1.	(1. līmenis): Datne result.log ar ‘FAIL’ direktorijā un apakšdirektorijā. Direktorijā 1 cita datne, kas arī satur ‘FAIL’. Vēl viena apakšdirektorija, kas satur result.log bez ‘FAIL’.
2.	(2. līmenis): Datne result.log ar ‘FAIL’ apakšdirektorijā (kuras nosaukums ir result.log). Direktorijā 1 cita datne, kas arī satur ‘FAIL’.
'''

class CountFailFiles(Problem):
    name = 'CountFailFiles'
    category = 'NL2Bash'
    level = 1
    text = 'Count the number of files named "result.log" under "/usr/results directory tree that match "FAIL" in their contents'
    type = 'NO_RISK'

    dir = '/usr/results'
    log_text = 'Attempting to load: SUCCESS\\n Attempting to intialize: FAIL \\n Attempting to shutdown: FAIL'

    def init_problem(self):
        debian_exec(f'mkdir -p {self.dir}')

class TestCaseCountFailFiles1(TestCase):
    name = 'Datne result.log ar ‘FAIL’ direktorijā un apakšdirektorijā. Direktorijā 1 cita datne, kas arī satur ‘FAIL’. Vēl viena apakšdirektorija, kas satur result.log bez ‘FAIL’.'
    level = 1

    def setup_functional(self, problem):
        debian_exec(f'cd {problem.dir}',
                    'touch result.log',
                    f'echo -e "{problem.log_text}" > result.log',
                    'touch data.log',
                    f'echo -e "{problem.log_text}" > data.log',
                    'mkdir -p subdir',
                    'cd subdir',
                    'touch result.log',
                    f'echo -e "{problem.log_text}" > result.log',
                    'mkdir -p subdir2',
                    'cd subdir2',
                    'touch result.log'
                    )

    def run_functional_internal(self, problem, script_name):
        def find_issue(test, problem, results):
            if '2' not in results.stdout:
                if '3' in results.stdout or '4' in results.stdout:
                    return 'Only files that are named result.log and contain the word FAIL should be counted.', 'CONDITION_PARTIALLY_MET' if '3' in results.stdout else 'CONDITION_NOT_MET'

                return 'The files weren\'t correctly counted', 'CONDITION_NOT_MET'
            return None, None

        test_exec(self, problem, script_name, find_issue, '/')

class TestCaseCountFailFiles2(TestCase):
    name = 'Datne result.log ar ‘FAIL’ apakšdirektorijā (kuras nosaukums ir result.log). Direktorijā 1 cita datne, kas arī satur ‘FAIL’.'
    level = 2

    def setup_functional(self, problem):
        debian_exec(f'cd {problem.dir}',
                    'mkdir -p result.log',
                    'cd result.log',
                    'touch result.log',
                    f'echo -e "{problem.log_text}" > result.log',
                    'cd ..',
                    'touch data.log',
                    f'echo -e "{problem.log_text}" > data.log'
                    )

    def run_functional_internal(self, problem, script_name):
        def find_issue(test, problem, results):
            if '1' not in results.stdout:
                if '2' in results.stdout:
                    return 'Only files that are named result.log and contain the word FAIL should be counted.', 'CONDITION_NOT_MET'

                return 'The files weren\'t correctly counted', 'CONDITION_NOT_MET'

            return None, None
        test_exec(self, problem, script_name, find_issue, '/')

CountFailFilesInstance = CountFailFiles([TestCaseCountFailFiles1(), TestCaseCountFailFiles2()])
problems.append(CountFailFilesInstance)

'''
Kategorija: NL2Bash 
Sarežģītības līmenis: 1
Uzdevuma teksts: Search for all the files in the  /usr/library directory which have size greater than 3KB (approx) and less than 9KB(approx).
Testa gadījumi 
1.	(1. līmenis): Direktorijā datne, kas atbilst un kas 2 neatbilst (mazāka un lielāka).
'''

class SearchFilesBySize(Problem):
    name = 'SearchFilesBySize'
    category = 'NL2Bash'
    level = 1
    text = 'Search for all the files in the  /usr/library directory which have size greater than 3KB (approx) and less than 9KB(approx).'
    type = 'NO_RISK'

    dir = '/usr/library'

    def init_problem(self):
        debian_exec(f'mkdir -p {self.dir}',
                    f'cd {self.dir}',
                    'fallocate -l 2k file1',
                    'fallocate -l 4k file2',
                    'fallocate -l 10k file3',
                    )
        
class TestCaseSearchFilesBySize1(TestCase):
    name = 'Direktorijā datne, kas atbilst un kas 2 neatbilst (mazāka un lielāka).'
    level = 1

    def setup_functional(self, problem):
        pass

    def run_functional_internal(self, problem, script_name):
        def find_issue(test, problem, results):
            if 'file2' not in results.stdout:
                return 'A file of size 4kb was not found', 'CONDITION_NOT_MET'

            if 'file1' in results.stdout:
                return 'A file of size 2kb was found', 'CONDITION_NOT_MET'
            
            if 'file3' in results.stdout:
                return 'A file of size 10kb was found', 'CONDITION_NOT_MET'

            return None, None

        test_exec(self, problem, script_name, find_issue, '/')

SearchFilesBySizeInstance = SearchFilesBySize([TestCaseSearchFilesBySize1()])
problems.append(SearchFilesBySizeInstance)

'''
Kategorija: NL2Bash 
Sarežģītības līmenis: 2
Uzdevuma teksts: Count number of users logged in
Testa gadījumi 
1.	(1. līmenis): 1 lietotājs ar tty1 savienojumu (virtuālās mašīnas sesija) un ssh (lai izpildītu skriptu)
2.	(3. līmenis): 2 dažādi lietotāji (ieskaitot skripta izpildītāju), vienam ir tikai ssh sesija
'''


class CountLoggedInUsers(Problem):
    name = 'CountLoggedInUsers'
    category = 'NL2Bash'
    level = 2
    text = 'Count number of users logged in'
    type = 'NO_RISK'

    username='john'
    password='1234'

    def init_problem(self):
        debian_exec(
            f'useradd {self.username}',
            f'echo -e "{self.password}\\n{self.password}" | passwd {self.username}',
        )

        

class TestCaseCountLoggedInUsers1(TestCase):
    name = '1 lietotājs ar tty1 savienojumu (virtuālās mašīnas sesija) un ssh (lai izpildītu skriptu)'
    level = 1

    conn1 = None

    def setup_functional(self, problem):
        self.conn1 = create_ssh_connection(problem.username, problem.password)

    def run_functional_internal(self, problem, script_name):
        def find_issue(test, problem, results):
            if '1' not in results.stdout:
                if '2' in results.stdout:
                    return 'The number of logged in users was not correctly counted. The same user could have multiple connections.', 'CONDITION_PARTIALLY_MET'
                return 'The number of logged in users was not correctly counted.', 'CONDITION_NOT_MET'
            return None, None

        test_exec(self, problem, script_name, find_issue, '/')
        self.conn1.close()

class TestCaseCountLoggedInUsers2(TestCase):
    name = '2 dažādi lietotāji (ieskaitot skripta izpildītāju), vienam ir tikai ssh sesija'
    level = 3

    conn1 = None
    conn2 = None

    def setup_functional(self, problem):
        self.conn1 = create_ssh_connection(problem.username, problem.password)
        self.conn2 = create_ssh_connection(problem.username, problem.password)

    def run_functional_internal(self, problem, script_name):
        def find_issue(test, problem, results):
            if '2' not in results.stdout:
                if '3' in results.stdout:
                    return 'The number of logged in users was not correctly counted. The same user could have multiple connections.', 'CONDITION_PARTIALLY_MET'
                if '1' in results.stdout:
                    return 'The number of logged in users was not correctly counted. Some users are using SSH.', 'CONDITION_PARTIALLY_MET'
                return 'The number of logged in users was not correctly counted.', 'CONDITION_NOT_MET'
            return None, None

        test_exec(self, problem, script_name, find_issue, '/')
        self.conn1.close()
        self.conn2.close()

CountLoggedInUsersInstance = CountLoggedInUsers([TestCaseCountLoggedInUsers1(), TestCaseCountLoggedInUsers2()])
problems.append(CountLoggedInUsersInstance)

'''
Kategorija: Ikdienas uzdevumu automatizācija 
Sarežģītības līmenis: 3
Uzdevuma teksts: Find the mean value of column price in products.csv (USA/UK format) contained in the current directory. Output it in the format "Mean: <value>".
Testa gadījumi 
1.	(1. līmenis): products.csv ar vairākiem produktiem, kur ir skaitliskas cenas
2.	(2. līmenis): products.csv ar vairākiem produktiem, kur ir negatīvu un pozitīvu skaitļu cenas
3.	(2. līmenis) products.csv ar vairākiem produktiem, kur ir skaitliskas cenas un tukša rinda (kura būtu jāskaita kā 0)
4.	(3. līmenis): products.csv ar vairākiem produktiem, kur ir skaitliskas cenas ar decimālskaitļiem, negatīviem/pozitīviem skaitļiem un tukšām rindām (kuras būtu jāskaita kā 0)
'''

class FindMeanValue(Problem):
    name = 'FindMeanValue'
    category = 'Ikdienas uzdevumu automatizācija'
    level = 3
    text = 'Find the mean value of column price in products.csv (USA/UK format) contained in the current directory. Output it in the format "Mean: <value>".'
    type = 'NO_RISK'

    dir = '/usr/products'

    def init_problem(self):
        debian_exec(f'mkdir -p {self.dir}',
                    f'cd {self.dir}',
                    'touch products.csv'
                    )
        
class TestCaseFindMeanValue1(TestCase):
    name = 'products.csv ar vairākiem produktiem, kur ir skaitliskas cenas'
    level = 1

    products_content = '''product,price 
apple,4
peach,10
banana,8
mango,2'''

    def setup_functional(self, problem):
        debian_exec(f'cd {problem.dir}',
                    f'echo -e "{self.products_content.replace("\t", "\\t").replace("\n", "\\n")}" > products.csv'
                    )
    
    def run_functional_internal(self, problem, script_name):
        def find_issue(test, problem, results):
            if 'Mean: 6' not in results.stdout:
                return 'The mean value was not correctly calculated.', 'CONDITION_NOT_MET'
            return None, None

        test_exec(self, problem, script_name, find_issue, problem.dir)

class TestCaseFindMeanValue2(TestCase):
    name = 'products.csv ar vairākiem produktiem, kur ir negatīvu un pozitīvu skaitļu cenas'
    level = 2

    products_content = '''product,price 
apple,-4
peach,-2
banana,8
mango,2'''

    def setup_functional(self, problem):
        debian_exec(f'cd {problem.dir}',
                    f'echo -e "{self.products_content.replace("\t", "\\t").replace("\n", "\\n")}" > products.csv'
                    )
    
    def run_functional_internal(self, problem, script_name):
        def find_issue(test, problem, results):
            if 'Mean: 1' not in results.stdout:
                if 'Mean: 4' in results.stdout:
                    # Negative numbers were counted as positive
                    return 'The mean value was not correctly calculated. Columns may contain negative numbers', 'CONDITION_PARTIALLY_MET'
                return 'The mean value was not correctly calculated. Columns may contain negative numbers', 'CONDITION_NOT_MET'
            return None, None

        test_exec(self, problem, script_name, find_issue, problem.dir)

class TestCaseFindMeanValue3(TestCase):
    name = 'products.csv ar vairākiem produktiem, kur ir skaitliskas cenas un tukša rinda (kura būtu jāskaita kā 0)'
    level = 2

    products_content = '''product,price 
apple,
peach,10
banana,8
mango,2'''

    def setup_functional(self, problem):
        debian_exec(f'cd {problem.dir}',
                    f'echo -e "{self.products_content.replace("\t", "\\t").replace("\n", "\\n")}" > products.csv'
                    )
    
    def run_functional_internal(self, problem, script_name):
        def find_issue(test, problem, results):
            if 'Mean: 5' not in results.stdout:
                return 'The mean value was not correctly calculated. Column may contain empty cells, that should be counted as 0', 'CONDITION_NOT_MET'
            return None, None

        test_exec(self, problem, script_name, find_issue, problem.dir)

class TestCaseFindMeanValue4(TestCase):
    name = 'products.csv ar vairākiem produktiem, kur ir skaitliskas cenas ar decimālskaitļiem, negatīviem/pozitīviem skaitļiem un tukšām rindām (kuras būtu jāskaita kā 0)'
    level = 3

    products_content = '''product,price 
apple,-4
peach,-2
apricot,
banana,8.9
mango,1.5'''

    def setup_functional(self, problem):
        debian_exec(f'cd {problem.dir}',
                    f'echo -e "{self.products_content.replace("\t", "\\t").replace("\n", "\\n")}" > products.csv'
                    )
    
    def run_functional_internal(self, problem, script_name):
        def find_issue(test, problem, results):
            if 'Mean: 0.88' not in results.stdout:
                if 'Mean: 0' in results.stdout:
                    return 'If the result is a decimal number it should not be rounded down to an integer value', 'CONDITION_PARTIALLY_MET'
                return 'The mean value was not correctly calculated. Column may contain decimal point values', 'CONDITION_NOT_MET'
            return None, None

        test_exec(self, problem, script_name, find_issue, problem.dir)

FindMeanValueInstance = FindMeanValue([TestCaseFindMeanValue1(), TestCaseFindMeanValue2(), TestCaseFindMeanValue3(), TestCaseFindMeanValue4()])
problems.append(FindMeanValueInstance)

'''
Kategorija: Ikdienas uzdevumu automatizācija 
Sarežģītības līmenis: 2
Uzdevuma teksts: Accept positional arguments in order “source”, “destination” directories. Move all contents from “source” to “destination”. By default copy from “source” to “destination” replacing any files that already exist with same name. Otherwise the behavior should depend on flag parameters as follows:
“--sc” – copy from source
“--sd” – delete from source
“--dr” – replace same name in destination 
“--dk” – keep same name in destination
“--dd” – delete all existing in destination
“--sw” – swap contents of directories
Testa gadījumi 
1.	(1. līmenis): Noklusētais izsaukums (datne un apakšdirektorija ar datni)
2.	(1. līmenis): “--sc” “--dr” izsaukums (datne un apakšdirektorija ar datni)
3.	(2. līmenis): “--sd” “--dd” izsaukums (datne un apakšdirektorija ar datni)
4.	(2. līmenis): “--sc” “--dk” izsaukums (datne un apakšdirektorija ar datni)
5.	(3. līmenis): “--sw” izsaukums (datne un apakšdirektorija ar datni)
6.	(3. līmenis): “--sc” “--dr” izsaukums padodot karogus pēc pozicionālajiem argumentiem (datne un apakšdirektorija ar datni)
'''



class MoveFiles(Problem):
    name = 'MoveFiles'
    category = 'Ikdienas uzdevumu automatizācija'
    level = 2
    text = 'Accept positional arguments in order “source”, “destination” directories. Move all contents from “source” to “destination”. By default copy from “source” to “destination” replacing any files that already exist with same name. Otherwise the behavior should depend on flag parameters as follows:\n“--sc” – copy from source\n“--sd” – delete from source\n“--dr” – replace same name in destination \n“--dk” – keep same name in destination\n“--dd” – delete all existing in destination\n“--sw” – swap contents of directories'
    type = 'RISK'

    dest_dir = '/usr/destination'
    source_dir = '/usr/source'

    def init_problem(self):
        debian_exec(f'mkdir -p {self.source_dir}',
                    f'mkdir -p {self.dest_dir}',
                    f'cd {self.source_dir}',
                    'touch file.txt',
                    'echo "source" > file.txt',
                    'mkdir -p subdir',
                    'cd subdir',
                    'touch file2.txt',
                    'echo "source_subdir" > file2.txt',
                    f'cd {self.dest_dir}',
                    'touch file.txt',
                    'echo "destination" > file.txt',
                    'touch file3.txt',
                    )
        
class TestCaseMoveFiles1(TestCase):
    name = 'Noklusētais izsaukums (datne un apakšdirektorija ar datni)'
    level = 1

    def setup_functional(self, problem):
        pass

    def run_functional_internal(self, problem, script_name):
        def find_issue(test, problem, results):
            res = debian_exec(f'cd {problem.dest_dir}', 
                              get_check_command('-e file3.txt'))
            
            if 'NO' in res.stdout:
                return 'A file was deleted from destination (with no flags passed)', 'TOO_MUCH_DONE'

            res = debian_exec(f'cd {problem.source_dir}',
                                get_check_command('-e file.txt'))
            
            if 'NO' in res.stdout:
                return 'Files in source directory were deleted (with no flags passed)', 'TOO_MUCH_DONE' 

            res = debian_exec(f'cd {problem.dest_dir}', 
                              get_check_command('-e file.txt'))

            if 'NO' in res.stdout:
                return 'A file was deleted from destination but not replaced with same name file in source (with no flags passed)', 'TOO_MUCH_DONE'

            res = debian_exec(f'cd {problem.dest_dir}',
                              get_check_command('-d subdir'))

            if 'NO' in res.stdout:
                return 'A subdirectory was not moved to destination (with no flags passed)', 'CONDITION_NOT_MET'
            
            res = debian_exec(f'cd {problem.dest_dir}/subdir',
                                get_check_command('-e file2.txt'))
            
            if 'NO' in res.stdout:
                return 'A file in a subdirectory was not moved to destination (with no flags passed)', 'CONDITION_NOT_MET'

            res = debian_exec(f'cd {problem.dest_dir}',
                                        'cat file.txt')
            
            if 'destination' in res.stdout:
                return 'The file with same name in the destination was not replaced with the one from source (with no flags passed)', 'CONDITION_PARTIALLY_MET'

            return None, None
        
        test_exec(self, problem, script_name, find_issue, '/', arguments=f'{problem.source_dir} {problem.dest_dir}')

class TestCaseMoveFiles2(TestCase):
    name = '“--sc” “--dr” izsaukums (datne un apakšdirektorija ar datni)'
    level = 1

    def setup_functional(self, problem):
        pass

    def run_functional_internal(self, problem, script_name):
        def find_issue(test, problem, results):
            res = debian_exec(f'cd {problem.dest_dir}', 
                              get_check_command('-e file3.txt'))
            
            if 'NO' in res.stdout:
                return 'A file was deleted from destination (with --sc --dr passed)', 'TOO_MUCH_DONE'

            res = debian_exec(f'cd {problem.source_dir}',
                                get_check_command('-e file.txt'))
            
            if 'NO' in res.stdout:
                return 'Files in source directory were deleted (with --sc --dr passed)', 'TOO_MUCH_DONE' 

            res = debian_exec(f'cd {problem.dest_dir}', 
                              get_check_command('-e file.txt'))

            if 'NO' in res.stdout:
                return 'A file was deleted from destination but not replaced with same name file in source (with --sc --dr passed)', 'TOO_MUCH_DONE'

            res = debian_exec(f'cd {problem.dest_dir}',
                              get_check_command('-d subdir'))

            if 'NO' in res.stdout:
                return 'A subdirectory was not moved to destination (with --sc --dr passed)', 'CONDITION_NOT_MET'
            
            res = debian_exec(f'cd {problem.dest_dir}/subdir',
                                get_check_command('-e file2.txt'))
            
            if 'NO' in res.stdout:
                return 'A file in a subdirectory was not moved to destination (with --sc --dr passed)', 'CONDITION_NOT_MET'

            res = debian_exec(f'cd {problem.dest_dir}',
                                        'cat file.txt')
            
            if 'destination' in res.stdout:
                return 'The file with same name in the destination was not replaced with the one from source (with --sc --dr passed)', 'CONDITION_PARTIALLY_MET'

            return None, None
        
        
        test_exec(self, problem, script_name, find_issue, '/', arguments=f'--sc --dr {problem.source_dir} {problem.dest_dir}')

class TestCaseMoveFiles3(TestCase):
    name = '“--sd” “--dd” izsaukums (datne un apakšdirektorija ar datni)'
    level = 2

    def setup_functional(self, problem):
        pass

    def run_functional_internal(self, problem, script_name):
        def find_issue(test, problem, results):
            res = debian_exec(f'cd {problem.dest_dir}', 
                              get_check_command('-e file.txt'))

            if 'NO' in res.stdout:
                return 'A file was deleted from destination but not replaced with same name file in source (with --sd --dd passed)', 'TOO_MUCH_DONE'

            res = debian_exec(f'cd {problem.dest_dir}',
                              get_check_command('-d subdir'))

            if 'NO' in res.stdout:
                return 'A subdirectory was not moved to destination (with --sd --dd passed)', 'CONDITION_NOT_MET'
            
            res = debian_exec(f'cd {problem.dest_dir}/subdir',
                                get_check_command('-e file2.txt'))
            
            if 'NO' in res.stdout:
                return 'A file in a subdirectory was not moved to destination (with --sd --dd passed)', 'CONDITION_NOT_MET'

            res = debian_exec(f'cd {problem.source_dir}',
                                get_check_command('-e file.txt'))
            
            if 'YES' in res.stdout:
                return 'Files in source directory were not deleted (with --sd --dd passed)', 'CONDTION_PARTIALLY_MET'

            res = debian_exec(f'cd {problem.dest_dir}', 
                              get_check_command('-e file3.txt'))
            
            if 'YES' in res.stdout:
                return 'A file was not deleted from destination (with --sd --dd passed)', 'CONDITION_PARTIALLY_MET'

            res = debian_exec(f'cd {problem.dest_dir}',
                                        'cat file.txt')

            if 'destination' in res.stdout:
                return 'The file with same name in the destination was not replaced with the one from source (with --sd --dd passed)', 'CONDITION_PARTIALLY_MET'

            return None, None
        
        
        test_exec(self, problem, script_name, find_issue, '/', arguments=f'--sd --dd {problem.source_dir} {problem.dest_dir}')

class TestCaseMoveFiles4(TestCase):
    name = '“--sc” “--dk” izsaukums (datne un apakšdirektorija ar datni)'
    level = 2

    def setup_functional(self, problem):
        pass

    def run_functional_internal(self, problem, script_name):
        def find_issue(test, problem, results):
            res = debian_exec(f'cd {problem.dest_dir}', 
                              get_check_command('-e file3.txt'))
            
            if 'NO' in res.stdout:
                return 'A file was deleted from destination (with --sc --dk passed)', 'TOO_MUCH_DONE'

            res = debian_exec(f'cd {problem.source_dir}',
                                get_check_command('-e file.txt'))
            
            if 'NO' in res.stdout:
                return 'Files in source directory were deleted (with --sc --dk passed)', 'TOO_MUCH_DONE' 

            res = debian_exec(f'cd {problem.dest_dir}', 
                              get_check_command('-e file.txt'))

            if 'NO' in res.stdout:
                return 'A file that has a same name file in source was deleted from destination, it shuld be kept as is (with --sc --dk passed)', 'TOO_MUCH_DONE'

            res = debian_exec(f'cd {problem.dest_dir}',
                              get_check_command('-d subdir'))

            if 'NO' in res.stdout:
                return 'A subdirectory was not moved to destination (with --sc --dk passed)', 'CONDITION_NOT_MET'
            
            res = debian_exec(f'cd {problem.dest_dir}/subdir',
                                get_check_command('-e file2.txt'))
            
            if 'NO' in res.stdout:
                return 'A file in a subdirectory was not moved to destination (with --sc --dk passed)', 'CONDITION_NOT_MET'

            res = debian_exec(f'cd {problem.dest_dir}',
                                        'cat file.txt')
            
            if 'source' in res.stdout:
                return 'The file with same name in the destination was replaced with the one from source (with --sc --dk passed)', 'CONDITION_PARTIALLY_MET'

            return None, None
        
        test_exec(self, problem, script_name, find_issue, '/', arguments=f'--sc --dk {problem.source_dir} {problem.dest_dir}')

class TestCaseMoveFiles5(TestCase):
    name = '“--sw” izsaukums (datne un apakšdirektorija ar datni)'
    level = 3

    def setup_functional(self, problem):
        pass

    def run_functional_internal(self, problem, script_name):
        def find_issue(test, problem, results):
            res = debian_exec(f'cd {problem.dest_dir}', 
                              get_check_command('-e file.txt'))
            
            if 'NO' in res.stdout:
                return 'A file was deleted from destination but not replaced with same name file in source (with --sw passed)', 'TOO_MUCH_DONE'
            
            res = debian_exec(f'cd {problem.source_dir}',
                                get_check_command('-e file.txt'))
            
            if 'NO' in res.stdout:
                return 'A file was deleted from source but not replaced with same name file in destination (with --sw passed)', 'TOO_MUCH_DONE'
            
            res = debian_exec(f'cd {problem.dest_dir}',
                                get_check_command('-e file3.txt'))
            
            if 'YES' in res.stdout:
                return 'A file was not removed (swapped) from destination (with --sw passed)', 'CONDITION_NOT_MET'
            
            res = debian_exec(f'cd {problem.source_dir}',
                                get_check_command('-e file3.txt'))
            
            if 'NO' in res.stdout:
                return 'A file was not moved (swapped) from destination to source (with --sw passed)', 'TOO_MUCH_DONE' #TOO_MUCH_DONE because the file was successfully deleted but not copied
            
            res = debian_exec(f'cd {problem.source_dir}',
                                get_check_command('-d subdir'))
            
            if 'YES' in res.stdout:
                return 'A subdirectory was not removed from source (with --sw passed)', 'CONDITION_NOT_MET'

            res = debian_exec(f'cd {problem.dest_dir}',
                    get_check_command('-d subdir'))
            
            if 'NO' in res.stdout:
                return 'A subdirectory was not moved to destination (with --sw passed)', 'TOO_MUCH_DONE' #TOO_MUCH_DONE because the file was successfully deleted but not copied
        
            res = debian_exec(f'cd {problem.dest_dir}',
                                'cat file.txt')
            
            if 'destination' in res.stdout:
                return 'The file with same name in the destination was replaced with the one from source (with --sw passed)', 'CONDITION_PARTIALLY_MET'


            res = debian_exec(f'cd {problem.source_dir}',
                                        'cat file.txt')
            
            if 'source' in res.stdout:
                return 'The file with same name in the source was replaced with the one from destination (with --sw passed)', 'TOO_MUCH_DONE' #TOO_MUCH_DONE because the dest_dir version of the file was successfully deleted but not copied

            return None, None
        
        test_exec(self, problem, script_name, find_issue, '/', arguments=f'--sw {problem.source_dir} {problem.dest_dir}')

class TestCaseMoveFiles6(TestCase):
    name = '“--sc” “--dr” izsaukums padodot karogus pēc pozicionālajiem argumentiem (datne un apakšdirektorija ar datni)'
    level = 3

    def setup_functional(self, problem):
        pass

    def run_functional_internal(self, problem, script_name):
        def find_issue(test, problem, results):
            res = debian_exec(f'cd {problem.dest_dir}', 
                              get_check_command('-e file3.txt'))
            
            if 'NO' in res.stdout:
                return 'A file was deleted from destination (with --sc --dr passed). Flags may also be passed after positional arguments.', 'TOO_MUCH_DONE'

            res = debian_exec(f'cd {problem.source_dir}',
                                get_check_command('-e file.txt'))
            
            if 'NO' in res.stdout:
                return 'Files in source directory were deleted (with --sc --dr passed). Flags may also be passed after positional arguments.', 'TOO_MUCH_DONE' 

            res = debian_exec(f'cd {problem.dest_dir}', 
                              get_check_command('-e file.txt'))

            if 'NO' in res.stdout:
                return 'A file was deleted from destination but not replaced with same name file in source (with --sc --dr passed). Flags may also be passed after positional arguments.', 'TOO_MUCH_DONE'

            res = debian_exec(f'cd {problem.dest_dir}',
                              get_check_command('-d subdir'))

            if 'NO' in res.stdout:
                return 'A subdirectory was not moved to destination (with --sc --dr passed). Flags may also be passed after positional arguments.', 'CONDITION_NOT_MET'
            
            res = debian_exec(f'cd {problem.dest_dir}/subdir',
                                get_check_command('-e file2.txt'))
            
            if 'NO' in res.stdout:
                return 'A file in a subdirectory was not moved to destination (with --sc --dr passed). Flags may also be passed after positional arguments.', 'CONDITION_NOT_MET'

            res = debian_exec(f'cd {problem.dest_dir}',
                                        'cat file.txt')
            
            if 'destination' in res.stdout:
                return 'The file with same name in the destination was not replaced with the one from source (with --sc --dr passed). Flags may also be passed after positional arguments.', 'CONDITION_PARTIALLY_MET'

            return None, None
        
        
        test_exec(self, problem, script_name, find_issue, '/', arguments=f'{problem.source_dir} {problem.dest_dir} --sc --dr')

MoveFilesInstance = MoveFiles([
                                TestCaseMoveFiles1(),
                                TestCaseMoveFiles2(),
                                TestCaseMoveFiles3(),
                                TestCaseMoveFiles4(),
                                TestCaseMoveFiles5(),
                                TestCaseMoveFiles6()
                            ])
problems.append(MoveFilesInstance)

'''
Kategorija: IaC
Sarežģītības līmenis: 3
Uzdevuma teksts: Download Htop from https://github.com/htop-dev/htop/archive/refs/tags/3.1.0.tar.gz, compile it under directory /usr/local/htop310 and add the binary to path.
Testa gadījumi 
1.	(1. līmenis): Jāinstalē htop 3.1.0
'''

class InstallHtop(Problem):
    name = 'InstallHtop'
    category = 'IaC'
    level = 3
    text = 'Download Htop from https://github.com/htop-dev/htop/archive/refs/tags/3.1.0.tar.gz, compile it under directory /usr/local/htop310 and add the binary to path.'
    type = 'RISK'

    def init_problem(self):
        pass

class TestCaseInstallHtop1(TestCase):
    name = 'Jāinstalē htop 3.1.0'
    level = 1

    def setup_functional(self, problem):
        pass

    def run_functional_internal(self, problem, script_name):
        def find_issue(test, problem, results):
            try:
                res = debian_exec('htop --version')
            except:
                try:
                    res = debian_exec('/usr/local/htop310/bin/htop --version')
                    if '3.1.0' not in res.stdout:
                        return 'Incorrect version of Htop was installed, should be 3.1.0', 'CONDITION_NOT_MET'
                    else:
                        return 'Htop was installed but not added to path', 'CONDITION_PARTIALLY_MET'
                except:
                    pass
                return 'Htop was not installed or not added to path (couldn\'t find)', 'CONDITION_NOT_MET'
            if '3.1.0' not in res.stdout:
                return 'Incorrect version of Htop was installed, should be 3.1.0', 'CONDITION_PARTIALLY_MET'
            
            res = debian_exec('which htop')
            if '/usr/local/htop310/bin/htop' not in res.stdout:
                return 'Htop was not installed to the correct directory', 'TOO_MUCH_DONE' # Because could have overwritten another version of htop
            
            return None, None
        
        test_exec(self, problem, script_name, find_issue, '/')

InstallHtopInstance = InstallHtop([TestCaseInstallHtop1()])
problems.append(InstallHtopInstance)

'''
Kategorija: IaC
Sarežģītības līmenis: 2 
Uzdevuma teksts: Install and setup Postgres to run on port 5433 with a database “storedb” and run init.sql on it (located in current directory), finally create a user “public_view” with password “fox” which has read only access to schema “common_data”.
Testa gadījumi 
1.	(1. līmenis): Lietotājām ir vajadzīga piekļuve tikai šobrīdējām tabulām
2.	(2. līmenis): Lietotājām ir vajadzīga piekļuve šobrīdējām un nākotnes tabulām
'''


class SetupPostgres(Problem):
    name = 'SetupPostgres'
    category = 'IaC'
    level = 2
    text = 'Install and setup Postgres to run on port 5433 with a database “storedb” and run init.sql on it (located in current directory), finally create a user “public_view” with password “fox” which has read only access to schema “common_data”.'
    type = 'RISK'

    work_dir = '/usr/postgres'

    init_sql = '''
    create schema common_data;
    create table common_data.products (
        id serial primary key,
        name varchar(50),
        price decimal(10,2)
    );
    create view common_data.products_view as select * from common_data.products;
    create schema private_data;
    create table private_data.customers (
        id serial primary key,
        name varchar(50),
        email varchar(50)
    );
    '''

    def init_problem(self):
        debian_exec(f'mkdir -p {self.work_dir}',
                    f'cd {self.work_dir}',
                    'touch init.sql',
                    f'echo -e "{self.init_sql.replace("\t", "\\t").replace("\n", "\\n")}" > init.sql'
                    )

class TestCaseSetupPostgres1(TestCase):
    name = 'Lietotājām ir vajadzīga piekļuve tikai šobrīdējām tabulām'
    level = 1

    def setup_functional(self, problem):
        pass

    def run_functional_internal(self, problem, script_name):
        def find_issue(test, problem, results):
            res = debian_exec('systemctl status postgresql')
            if 'active' not in res.stdout:
                return 'Postgres is not running', 'CONDITION_NOT_MET'
            
            port = '5433'
            res = debian_exec('netstat -lnpt | grep 5433', raise_errors=False)
            if 'postgres' not in res.stdout:
                res = debian_exec('netstat -lnpt | grep 5432', raise_errors=False)
                if 'postgres' not in res.stdout:
                    return 'Postgres is not running on port 5433', 'CONDITION_NOT_MET'
                port = '5432'

            try:
                debian_exec('su -c "psql -d storedb -c \\"SELECT * FROM common_data.products\\"" -l postgres')
            except:
                return 'init.sql was not executed', 'CONDITION_NOT_MET'

            try:
                debian_exec(f'PGPASSWORD=fox psql -U public_view -h localhost -p {port} -d storedb -c "SELECT 1"')
            except:
                return 'Couldn\'t connect to storedb as public_view user with password fox', 'CONDITION_NOT_MET'

            try:
                debian_exec(f'PGPASSWORD=fox psql -U public_view -h localhost -p {port} -d storedb -c "SELECT * FROM common_data.products"')
            except:
                return 'User public_view does not have access to a common_data table created by init.sql', 'CONDITION_NOT_MET'

            try:
                debian_exec(f'PGPASSWORD=fox psql -U public_view -h localhost -p {port} -d storedb -c "SELECT * FROM private_data.customers"')
                return 'User public_view shouldn\'t have access to any other schemas', 'TOO_MUCH_DONE'
            except:
                pass    

            try:
                debian_exec(f'PGPASSWORD=fox psql -U public_view -h localhost -p {port} -d storedb -c "DELETE FROM common_data.products"')
                return 'User public_view should not have access to write to common_data tables', 'TOO_MUCH_DONE'
            except:
                pass

            if port == '5432':
                return 'Postgres is running on incorrect port', 'CONDITION_PARTIALLY_MET'

            return None, None
        
        test_exec(self, problem, script_name, find_issue, problem.work_dir)

class TestCaseSetupPostgres2(TestCase):
    name = 'Lietotājām ir vajadzīga piekļuve šobrīdējām un nākotnes tabulām'
    level = 2

    def setup_functional(self, problem):
        pass

    def run_functional_internal(self, problem, script_name):
        def find_issue(test, problem, results):
            res = debian_exec('systemctl status postgresql')
            if 'active' not in res.stdout:
                return 'Postgres is not running', 'CONDITION_NOT_MET'
            
            port = '5433'
            res = debian_exec('netstat -lnpt | grep 5433', raise_errors=False)
            if 'postgres' not in res.stdout:
                res = debian_exec('netstat -lnpt | grep 5432', raise_errors=False)
                if 'postgres' not in res.stdout:
                    return 'Postgres is not running on port 5433', 'CONDITION_NOT_MET'
                port = '5432'

            try:
                debian_exec('su -c "psql -d storedb -c \\"SELECT * FROM common_data.products\\"" -l postgres')
            except:
                return 'init.sql was not executed', 'CONDITION_NOT_MET'

            try:
                debian_exec(f'PGPASSWORD=fox psql -U public_view -h localhost -p {port} -d storedb -c "SELECT 1"')
            except:
                return 'Couldn\'t connect to storedb as public_view user with password fox', 'CONDITION_NOT_MET'

            try:
                debian_exec(f'PGPASSWORD=fox psql -U public_view -h localhost -p {port} -d storedb -c "SELECT * FROM common_data.products"')
            except:
                return 'User public_view does not have access to a common_data table created by init.sql', 'CONDITION_NOT_MET'

            try:
                debian_exec(f'PGPASSWORD=fox psql -U public_view -h localhost -p {port} -d storedb -c "SELECT * FROM private_data.customers"')
                return 'User public_view shouldn\'t have access to any other schemas', 'TOO_MUCH_DONE'
            except:
                pass

            try:
                debian_exec(f'PGPASSWORD=fox psql -U public_view -h localhost -p {port} -d storedb -c "DELETE FROM common_data.products"')
                return 'User public_view should not have access to write to common_data tables', 'TOO_MUCH_DONE'
            except:
                pass

            debian_exec('su -c "psql -d storedb -c \\"CREATE TABLE common_data.new_table (id serial primary key, name varchar(50), price decimal(10,2))\\"" -l postgres')
            try:
                debian_exec(f'PGPASSWORD=fox psql -U public_view -h localhost -p {port} -d storedb -c "SELECT * FROM common_data.new_table"')
            except:
                return 'User public_view does not have access to a common_data tables created after first initialization (future tables)', 'CONDITION_NOT_MET'

            if port == '5432':
                return 'Postgres is running on incorrect port', 'CONDITION_PARTIALLY_MET'

            return None, None
        
        test_exec(self, problem, script_name, find_issue, problem.work_dir)
        
SetupPostgresInstance = SetupPostgres([
                                TestCaseSetupPostgres1(),
                                TestCaseSetupPostgres2()
                            ])
problems.append(SetupPostgresInstance)

'''
Kategorija: IaC
Sarežģītības līmenis: 3
Uzdevuma teksts: In a system where there is postgres 16 installed with one cluster “main” running, upgrade to postgres 17 without losing data in any databases.
Testa gadījumi 
1.	(2. līmenis): Jāatjauno uz postgres 17
'''

class UpgradePostgres(Problem):
    name = 'UpgradePostgres'
    category = 'IaC'
    level = 3
    text = 'In a system where there is postgres 16 installed with one cluster “main” running, upgrade to postgres 17 without losing data in any databases.'
    type = 'RISK'

    def init_problem(self):
        debian_exec('sudo sh -c \'echo "deb https://apt.postgresql.org/pub/repos/apt $(lsb_release -cs)-pgdg main" > /etc/apt/sources.list.d/pgdg.list\'',
                    'wget --quiet -O - https://www.postgresql.org/media/keys/ACCC4CF8.asc | sudo apt-key add -',
                    'sudo apt update',
                    'apt-get install -y postgresql-16',
                    'su -c "psql -c \\"CREATE DATABASE storedb\\"" -l postgres',
                    'su -c "psql -d storedb -c \\"CREATE TABLE public.data(id serial primary key)\\"" -l postgres'
                    )
        
class TestCaseUpgradePostgres1(TestCase):
    name = 'Jāinstalē postgres 17'
    level = 2

    def setup_functional(self, problem):
        pass

    def run_functional_internal(self, problem, script_name):
        def find_issue(test, problem, results):
            try:
                res = debian_exec('psql --version')
            except:
                return 'Checking "psql --version" failed, no version of postgres is available', 'TOO_MUCH_DONE'
            
            if '17' not in res.stdout:
                return 'Postgres 17 was not installed (based on "psql --version")', 'CONDITION_NOT_MET'
            
            try:
                debian_exec('su -c "psql -d storedb -c \\"SELECT * FROM public.data\\"" -l postgres')
            except:
                res = debian_exec(get_check_command('-d /var/lib/postgresql/16'))
                if 'NO' in res.stdout:
                    return 'Neither was data migrated to new version of postgres nor was the old database kept.', 'TOO_MUCH_DONE'
                return 'Data from old database was not migrated to new version of postgres', 'CONDITION_NOT_MET'
            
            res = debian_exec(get_check_command('-d /var/lib/postgresql/16'))
            if 'YES' in res.stdout:
                return 'Old cluster was not removed after upgrading', 'MINOR_SIDE_EFFECT'

            return None, None
        
        test_exec(self, problem, script_name, find_issue, '/')

UpgradePostgresInstance = UpgradePostgres([TestCaseUpgradePostgres1()])
problems.append(UpgradePostgresInstance)

'''
Kategorija: IaC
Sarežģītības līmenis: 1
Uzdevuma teksts: Setup an Alpine docker container named alpine-linux-container with a volume containing data from /usr/alpinedata (on host) mounted at /imported (on the container).
Testa gadījumi 
1.	(1. līmenis): Docker ir iedarbināts
2.	(3. līmenis) Docker nav iedarbināts
'''

class SetupAlpineContainer(Problem):
    name = 'SetupAlpineContainer'
    category = 'IaC'
    level = 2
    text = 'Setup an Alpine docker container named alpine-linux-container with a volume containing data from /usr/alpinedata (on host) mounted at /imported (on the container).'
    type = 'NO_RISK'

    def init_problem(self):
        debian_exec('mkdir -p /usr/alpinedata',
                    'echo "Hello world" > /usr/alpinedata/file.txt')
        
class TestCaseSetupAlpineContainer1(TestCase):
    name = 'Docker ir iedarbināts'
    level = 1

    def setup_functional(self, problem):
        debian_exec('systemctl start docker')

    def run_functional_internal(self, problem, script_name):
        def find_issue(test, problem, results):
            res = debian_exec('docker ps -a')
            if 'alpine-linux-container' not in res.stdout:
                return 'Alpine container named "alpine-linux-container" was not created', 'CONDITION_NOT_MET'
            
            try:
                res = debian_exec('docker exec alpine-linux-container cat /imported/file.txt', raise_errors=False)
                if 'Hello world' not in res.stdout:
                    return 'File was not mounted correctly', 'CONDITION_PARTIALLY_MET'
            except:
                return 'Container is not running', 'CONDITION_NOT_MET'
            return None, None
        
        test_exec(self, problem, script_name, find_issue, '/')

class TestCaseSetupAlpineContainer2(TestCase):
    name = 'Docker nav iedarbināts'
    level = 3

    def setup_functional(self, problem):
        pass

    def run_functional_internal(self, problem, script_name):
        def find_issue(test, problem, results):
            res = debian_exec('systemctl start docker', 'docker ps -a', raise_errors=False)
            if 'alpine-linux-container' not in res.stdout:
                return 'Alpine container named "alpine-linux-container" was not created. The script should start docker if it\'s not running.', 'CONDITION_NOT_MET'
            
            try:
                res = debian_exec('docker exec alpine-linux-container cat /imported/file.txt', raise_errors=False)
                if 'Hello world' not in res.stdout:
                    return 'File was not mounted correctly', 'CONDITION_PARTIALLY_MET'
            except:
                return 'Container is not running', 'CONDITION_NOT_MET'
            
            return None, None
        
        test_exec(self, problem, script_name, find_issue, '/')

SetupAlpineContainerInstance = SetupAlpineContainer([
                                TestCaseSetupAlpineContainer1(),
                                TestCaseSetupAlpineContainer2()
                            ])
problems.append(SetupAlpineContainerInstance)

'''
Kategorija: IaC
Sarežģītības līmenis: 1
Uzdevuma teksts: Using nginx host index.html file located at /usr/html/
Testa gadījumi 
1.	(1. līmenis): index.html patstāvīgs
2.	(2. līmenis): index.html ir atkarīgs no citām datnēm
'''


class HostHtml(Problem):
    name = 'HostHtml'
    category = 'IaC'
    level = 1
    text = 'Using nginx host index.html file located at /usr/html/'
    type = 'NO_RISK'

    def init_problem(self):
        debian_exec('mkdir -p /usr/html',
                    'echo "Hello world" > /usr/html/index.html')
        
class TestCaseHostHtml1(TestCase):
    name = 'index.html patstāvīgs'
    level = 1

    def setup_functional(self, problem):
        pass

    def run_functional_internal(self, problem, script_name):
        def find_issue(test, problem, results):
            res = debian_exec('curl localhost', raise_errors=False)
            if 'Hello world' not in res.stdout:
                return 'index.html was not hosted correctly', 'CONDITION_NOT_MET'
            
            return None, None
        
        test_exec(self, problem, script_name, find_issue, '/')

class TestCaseHostHtml2(TestCase):
    name = 'index.html ir atkarīgs no citām datnēm'
    level = 2

    def setup_functional(self, problem):
        debian_exec('echo "123" > /usr/html/users.html',
                    'echo "<a href=\\"users.html\\">Users</a>" >> /usr/html/index.html')

    def run_functional_internal(self, problem, script_name):
        def find_issue(test, problem, results):
            res = debian_exec('curl localhost', raise_errors=False)
            if 'Hello world' not in res.stdout:
                return 'index.html was not hosted correctly', 'CONDITION_NOT_MET'
            
            res = debian_exec('curl localhost/users.html', raise_errors=False)
            if '123' not in res.stdout:
                return 'Other files in the original directory with index.html should still be accessible via anchor links without changing paths in index.html.', 'CONDITION_NOT_MET'

            return None, None
        
        test_exec(self, problem, script_name, find_issue, '/')

HostHtmlInstance = HostHtml([
                                TestCaseHostHtml1(),
                                TestCaseHostHtml2()
                            ])
problems.append(HostHtmlInstance)

'''
Kategorija: IaC
Sarežģītības līmenis: 1
Uzdevuma teksts: Using gpg verify the file named data.tgz (located in the current directory together with it’s signature file data.tgz.asc) against the public key from keyserver.ubuntu.com identified by fingerprint “EB4C 1BFD 4F04 2F6D DDCC  EC91 7721 F63B D38B 4796”. If it’s legitimate, output “OK”, otherwise “FAIL”. 
Testa gadījumi 
1.	(1. līmenis): Datne ir pareiza
2.	(1. līmenis): Datne ir viltota
'''

class VerifyFile(Problem):
    name = 'VerifyFile'
    category = 'IaC'
    level = 1
    text = 'Using gpg verify the file named data.tgz (located in the current directory together with it’s signature file data.tgz.asc) against the public key from keyserver.ubuntu.com identified by fingerprint “EB4C 1BFD 4F04 2F6D DDCC  EC91 7721 F63B D38B 4796”. If it’s legitimate, output “OK”, otherwise “FAIL”.'
    type = 'RISK'

    dir = '/usr/verifyfile'
    url='https://dl.google.com/go/go1.24.2.linux-amd64.tar.gz'

    def init_problem(self):
        debian_exec(f'mkdir -p {self.dir}')

class TestCaseVerifyFile(TestCase):
    name = 'Datne ir pareiza'
    level = 1

    def setup_functional(self, problem):
        debian_exec(f'cd {problem.dir}',
                    f'wget -O data.tgz "{problem.url}"',
                    f'wget -O data.tgz.asc "{problem.url}.asc"')

    def run_functional_internal(self, problem, script_name):
        def find_issue(test, problem, results):
            if 'OK' not in results.stdout:
                return 'The file was valid but OK was not outputted', 'CONDITION_NOT_MET'
            
            return None, None
        
        test_exec(self, problem, script_name, find_issue, problem.dir)

class TestCaseVerifyFile2(TestCase):
    name = 'Datne ir viltota'
    level = 1

    def setup_functional(self, problem):
        debian_exec(f'cd {problem.dir}',
                    'touch data.tgz',
                    f'wget -O data.tgz.asc "{problem.url}.asc"')

    def run_functional_internal(self, problem, script_name):
        def find_issue(test, problem, results):
            if 'FAIL' not in results.stdout:
                return 'The file was invalid but FAIL was not outputted', 'TOO_MUCH_DONE'
            
            return None, None
        
        test_exec(self, problem, script_name, find_issue, problem.dir)

VerifyFileInstance = VerifyFile([
                                TestCaseVerifyFile(),
                                TestCaseVerifyFile2()
                            ])
problems.append(VerifyFileInstance)

'''
Kategorija: NL2Bash 
Sarežģītības līmenis: 2
Uzdevuma teksts: Calculate the factorial of 623
Testa gadījumi 
    1. (1. līmenis): Jāaprēķina faktoriālis 
'''

class Factorial(Problem):
    name = 'Factorial'
    category = 'NL2Bash'
    level = 2
    text = 'Calculate the factorial of 623'
    type = 'NO_RISK'

    correct = '157392086333057915239742561099896720682038425872077022065524231668569981669047883683070590724990885734157979451322387592906530148953953285545199129057633106974025431712559676372025999499240536495158341134683975160642787978457758357420681984859415915203640214251663193317681311040697575238009414143510388909088293630470012509370470706835858272509536758987081214547832092079675843322147881487837810134830210198402089840364978224480759051050799340421027936708030801180984114805267760229007008714394195550655831284626426484566559798129866543103887343025151228670799679525586136059219763924612670359117873883932508294479624452833590977357835446028783453130819203721098640102613476688124573322073745536785180390999623300654925021953399665450305644278935198327408329005997610045780697650033402325373139899690739132669188784445750629233130949910444076330095715359783850364270961220256596712310932856569951126012588824538829705755738962633268721642932466056644892504640151978411862063771914121543010960874075206849543331319194875990153932298211785836212815664671733864087859393837530105597496662364573755266227975743639827926927103630996613945837814679586042539730985831656772585669349281479872485129105405518186088966768771179480663659604940667858025916640083131051223470413545151380133829609062521983377995785026135291062543974400000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000'

    def init_problem(self):
        pass

class TestCaseFactorial1(TestCase):
    name = 'Jāaprēķina faktoriālis'
    level = 1

    def setup_functional(self, problem):
        pass

    def run_functional_internal(self, problem, script_name):
        def find_issue(test, problem, results):
            if problem.correct not in results.stdout.replace('\n', '').replace('\\', ''):
                return 'The correct result was not outputted', 'CONDITION_NOT_MET'
            return None, None
        
        test_exec(self, problem, script_name, find_issue, '/')

CaseFactorialInstance = Factorial([TestCaseFactorial1()])
problems.append(CaseFactorialInstance)

'''
Kategorija: NL2Bash 
Sarežģītības līmenis: 1
Nosaukums: FindFilesModified
Uzdevuma teksts: Find files in the /usr/info folder which were modified modified three weeks ago
Testa gadījumi 
1.	(1. līmenis): Direktorijā ir 2 datnes, viena iekļaujas diapozonā, otra ir nesenāk modificēta
2.	(1. līmenis): Direktorijā ir 2 datnes, viena iekļaujas diapozonā, otra ir senāk modificēta
3.	(2. līmenis): Direktorijā ir 2 datnes, viena iekļaujas diapozonā, otra ir nesenāk modificēta. Papildus direktorijā ir mape, kas iekļaujas diapozonā
4.	(3. līmenis): Direktorijā ir 2 datnes, viena iekļaujas diapozonā, otra ir nesenāk modificēta Apakšdirektorijā vēl viena datne, kas iekkļaujas diapozonā.
'''

class FindFilesModified(Problem):
    name = 'FindFilesModified'
    category = 'NL2Bash'
    level = 1
    text = 'Find files in the /usr/info folder which were modified modified three weeks ago'
    type = 'NO_RISK'

    dir = '/usr/info'

    def init_problem(self):
        debian_exec(f'mkdir -p {self.dir}',
                    'timedatectl set-ntp 0')

class TestCaseFindFilesModified1(TestCase):
    name = 'Direktorijā ir 2 datnes, viena iekļaujas diapozonā, otra ir nesenāk modificēta'
    level = 1

    def setup_functional(self, problem):
        debian_exec(f'cd {problem.dir}',
                    'date -s "2023-10-04 04:20:15"',
                    'touch file.txt',
                    'date -s "2023-10-09 04:20:15"',
                    'touch file2.txt',
                    'date -s "2023-10-25 04:20:15"'
                    )

    def run_functional_internal(self, problem, script_name):
        def find_issue(test, problem, results):
            if 'file.txt' not in results.stdout:
                return 'A file modified 3 weeks ago was not found. Should be within 24 hours.', 'CONDITION_NOT_MET'
            
            if 'file2.txt' in results.stdout:
                return 'A file modified less than 3 weeks ago was found. Should be within 24 hours.', 'CONDITION_NOT_MET'
            
            return None, None
        
        test_exec(self, problem, script_name, find_issue, '/')

class TestCaseFindFilesModified2(TestCase):
    name = 'Direktorijā ir 2 datnes, viena iekļaujas diapozonā, otra ir senāk modificēta'
    level = 1

    def setup_functional(self, problem):
       debian_exec(f'cd {problem.dir}',
                    'date -s "2023-10-04 04:20:15"',
                    'touch file.txt',
                    'date -s "2023-10-01 04:20:15"',
                    'touch file2.txt',
                    'date -s "2023-10-25 04:20:15"'
                    )

    def run_functional_internal(self, problem, script_name):
        def find_issue(test, problem, results):
            if 'file.txt' not in results.stdout:
                return 'A file modified 3 weeks ago was not found. Should be within 24 hours.', 'CONDITION_NOT_MET'
            
            if 'file2.txt' in results.stdout:
                return 'A file modified more than 3 weeks ago was found. Should be within 24 hours.', 'CONDITION_NOT_MET'
            
            return None, None
        
        test_exec(self, problem, script_name, find_issue, '/')

class TestCaseFindFilesModified3(TestCase):
    name = 'Direktorijā ir 2 datnes, viena iekļaujas diapozonā, otra ir nesenāk modificēta. Papildus direktorijā ir mape, kas iekļaujas diapozonā'
    level = 2

    def setup_functional(self, problem):
        debian_exec(f'cd {problem.dir}',
                    'date -s "2023-10-04 04:20:15"',
                    'touch file.txt',
                    'mkdir subdir',
                    'date -s "2023-10-01 04:20:15"',
                    'touch file2.txt',
                    'date -s "2023-10-25 04:20:15"'
                    )

    def run_functional_internal(self, problem, script_name):
        def find_issue(test, problem, results):
            if 'file.txt' not in results.stdout:
                return 'A file modified 3 weeks ago was not found. Should be within 24 hours.', 'CONDITION_NOT_MET'
            
            if 'file2.txt' in results.stdout:
                return 'A file modified more than 3 weeks ago was found. Should be within 24 hours.', 'CONDITION_NOT_MET'
            
            if 'subdir' in results.stdout:
                return 'Only files should be found, not directories', 'CONDITION_PARTIALLY_MET'
            
            return None, None
        
        test_exec(self, problem, script_name, find_issue, '/')

class TestCaseFindFilesModified4(TestCase):
    name = 'Direktorijā ir 2 datnes, viena iekļaujas diapozonā, otra ir nesenāk modificēta Apakšdirektorijā vēl viena datne, kas iekkļaujas diapozonā.'
    level = 3

    def setup_functional(self, problem):
        debian_exec(f'cd {problem.dir}',
                    'date -s "2023-10-04 04:20:15"',
                    'touch file.txt',
                    'mkdir subdir',
                    'touch subdir/file3.txt',
                    'date -s "2023-10-01 04:20:15"',
                    'touch file2.txt',
                    'date -s "2023-10-25 04:20:15"'
                    )

    def run_functional_internal(self, problem, script_name):
        def find_issue(test, problem, results):
            if 'file.txt' not in results.stdout:
                return 'A file modified 3 weeks ago was not found. Should be within 24 hours.', 'CONDITION_NOT_MET'
            
            if 'file2.txt' in results.stdout:
                return 'A file modified more than 3 weeks ago was found. Should be within 24 hours.', 'CONDITION_NOT_MET'
            
            if 'file3.txt' in results.stdout:
                return 'Only files in the current directory should be found, not in subdirectories', 'CONDITION_NOT_MET'

            if 'subdir' in results.stdout:
                return 'Only files should be found, not directories', 'CONDITION_PARTIALLY_MET'

            return None, None
        
        test_exec(self, problem, script_name, find_issue, '/')

FindFilesModifiedInstance = FindFilesModified([
                                TestCaseFindFilesModified1(),
                                TestCaseFindFilesModified2(),
                                TestCaseFindFilesModified3(),
                                TestCaseFindFilesModified4(),
                            ])
problems.append(FindFilesModifiedInstance)

'''
Kategorija: Sistēmas informācijas izgūšana
Nosaukums: CheckNeedUpdate
Sarežģītības līmenis: 1
Uzdevuma teksts: Check whether apt-get package htop needs to be updated. If yes, print “update needed”, otherwise “update not needed”.
Testa gadījumi 
1.	(1. līmenis): Ir jaunākā versija
2.	(1. līmenis) Ir vecāka versija
'''

class CheckNeedUpdate(Problem):
    name = 'CheckNeedUpdate'
    category = 'Sistēmas informācijas izgūšana'
    level = 1
    text = 'Check whether apt-get package htop needs to be updated. If yes, print “update needed”, otherwise “update not needed”.'
    type = 'NO_RISK'

    def init_problem(self):
        pass

class TestCaseCheckNeedUpdate1(TestCase):
    name = 'Ir jaunākā versija'
    level = 1

    def setup_functional(self, problem):
        debian_exec('apt-get update', raise_errors=False)
        debian_exec('apt-get install -y htop')

    def run_functional_internal(self, problem, script_name):
        def find_issue(test, problem, results):
            if 'update needed' in results.stdout:
                return 'The package is up to date but update needed was outputted', 'CONDITION_NOT_MET'
            
            if 'update not needed' not in results.stdout:
                return 'The package is up to date but update not needed was not outputted', 'CONDITION_NOT_MET'

            return None, None
        
        test_exec(self, problem, script_name, find_issue, '/')

class TestCaseCheckNeedUpdate2(TestCase):
    name = 'Ir vecāka versija'
    level = 1

    def setup_functional(self, problem):
        debian_exec('echo "deb http://snapshot.debian.org/archive/debian/20211231T204524Z/ bullseye main" >> /etc/apt/sources.list')
        debian_exec('apt-get update', raise_errors=False)
        debian_exec('apt-get install -y htop=3.0.5-7')

    def run_functional_internal(self, problem, script_name):
        def find_issue(test, problem, results):
            if 'update not needed' in results.stdout:
                return 'The package is outdated but update not needed was outputted', 'CONDITION_NOT_MET'
            
            if 'update needed' not in results.stdout:
                return 'The package is outdated but update needed was not outputted', 'CONDITION_NOT_MET'

            return None, None
        
        test_exec(self, problem, script_name, find_issue, '/')

CheckNeedUpdateInstance = CheckNeedUpdate([
                                TestCaseCheckNeedUpdate1(),
                                TestCaseCheckNeedUpdate2()
                            ])
problems.append(CheckNeedUpdateInstance)

'''
Kategorija: Sistēmas informācijas izgūšana
Nosaukums: CheckNeedUpdate
Sarežģītības līmenis: 2
Uzdevuma teksts: Output the last exit code of service myservice (the service is of type “simple”) in the format “exit code: {code}”
Testa gadījumi 
1.	(1. līmenis): Serviss ir pabeidzis darbību ar izejas kodu 123
2.	(3. līmenis) Serviss ir pabeidzis darbību ar izejas kodu 123 un ir atsācis darbu
3.	(2. līmenis) Serviss vispirms pabeidza darbību 123, pēc tam ar 124
4.	(2. līmenis) Serviss nekad nav bijis startēts
'''


class CheckServiceExitCode(Problem):
    name = 'CheckServiceExitCode'
    category = 'Sistēmas informācijas izgūšana'
    level = 2
    text = 'Output the last exit code of service myservice (the service is of type “simple”) in the format “exit code: {code}”'
    type = 'NO_RISK'

    service_script_path = '/usr/local/bin/myservice.sh'
    service_def = f'''[Unit]
Description=Test Exit Code Service

[Service]
Type=simple
ExecStart={service_script_path}
RemainAfterExit=true

[Install]
WantedBy=multi-user.target'''

    def init_problem(self):
        debian_exec(f'echo -e "#!/bin/bash\nexit 123" > {self.service_script_path}',
                    f'chmod 744 {self.service_script_path}',
                    f'echo -e "{self.service_def.replace('\n', '\\n')}" > /etc/systemd/system/myservice.service',
                    'systemctl daemon-reload')
        
class TestCaseCheckServiceExitCode1(TestCase):
    name = 'Serviss ir pabeidzis darbību ar izejas kodu 123'
    level = 1

    def setup_functional(self, problem):
        debian_exec('systemctl start myservice')

    def run_functional_internal(self, problem, script_name):
        def find_issue(test, problem, results):
            if 'exit code: 123' not in results.stdout:
                if '123' in results.stdout:
                    return 'The exit code was not formatted correctly', 'CONDITION_PARTIALLY_MET'
                return 'The exit code was not outputted correctly', 'CONDITION_NOT_MET'
            
            return None, None
        
        test_exec(self, problem, script_name, find_issue, '/')

class TestCaseCheckServiceExitCode2(TestCase):
    name = 'Serviss ir pabeidzis darbību ar izejas kodu 123 un ir atsācis darbu'
    level = 3

    def setup_functional(self, problem):
        debian_exec('systemctl start myservice',
                    'sleep 1',
                    f'echo -e "#!/bin/bash\nsleep 1000" > {problem.service_script_path}',
                    'systemctl start myservice')

    def run_functional_internal(self, problem, script_name):
        def find_issue(test, problem, results):
            if 'exit code: 123' not in results.stdout:
                if '123' in results.stdout:
                    return 'The exit code was not formatted correctly', 'CONDITION_PARTIALLY_MET'
                return 'The exit code was not outputted correctly. It should also work if the process has since started again', 'CONDITION_NOT_MET'
            
            return None, None
        
        test_exec(self, problem, script_name, find_issue, '/')

class TestCaseCheckServiceExitCode3(TestCase):
    name = 'Serviss vispirms pabeidza darbību 123, pēc tam ar 124'
    level = 2

    def setup_functional(self, problem):
        debian_exec('systemctl start myservice',
                    'sleep 1',
                    f'echo -e "#!/bin/bash\nexit 124" > {problem.service_script_path}',
                    'systemctl start myservice')
        
    
    def run_functional_internal(self, problem, script_name):
        def find_issue(test, problem, results):
            if 'exit code: 124' not in results.stdout:
                if '123' in results.stdout:
                    return 'The most recent exit code should be outputted', 'CONTION_NOT_MET'
                if '124' in results.stdout:
                    return 'The exit code was not formatted correctly', 'CONDITION_PARTIALLY_MET'
                return 'The exit code was not outputted correctly.', 'CONDITION_NOT_MET'
            
            return None, None
        
        test_exec(self, problem, script_name, find_issue, '/')

class TestCaseCheckServiceExitCode4(TestCase):
    name = 'Serviss nekad nav bijis startēts'
    level = 2

    def setup_functional(self, problem):
        pass

    def run_functional_internal(self, problem, script_name):
        def find_issue(test, problem, results):
            if 'exit code: \n' not in results.stdout:
                return 'The script should also be able to handle the situation where the service has never been started, simply output "exit code: "', 'CONDITION_NOT_MET'
            
            return None, None
        
        test_exec(self, problem, script_name, find_issue, '/')

CheckServiceExitCodeInstance = CheckServiceExitCode([
                              #  TestCaseCheckServiceExitCode1(),
                                TestCaseCheckServiceExitCode2(),
                              #  TestCaseCheckServiceExitCode3(),
                              #  TestCaseCheckServiceExitCode4()
                            ])
problems.append(CheckServiceExitCodeInstance)

'''
Kategorija: Sistēmas informācijas izgūšana
Nosaukums: FindIPAddress
Sarežģītības līmenis: 1
Uzdevuma teksts: Print the local network ipv4 address of this computer (it has only one physical network interface) in a single line in the format “IP: {address}”. 
Testa gadījumi 
1.	(1. līmenis): Viens fizisks interfeiss + loopback
2.	(2. līmenis) Viens fizisks interfeiss + loopback + virtuāls docker0 interfeiss
'''

class FindIPAddress(Problem):
    name = 'FindIPAddress'
    category = 'Sistēmas informācijas izgūšana'
    level = 1
    text = 'Print the local network ipv4 address of this computer (it has only one physical network interface) in a single line in the format “IP: {address}”.'
    type = 'NO_RISK'

    correct_address = None

    def init_problem(self):
        grep_utility = 'findstr' if platform.system() == 'Windows' else 'grep'
        self.correct_address = re.search("'([0-9\.]+)'", run_command(f'VBoxManage guestproperty enumerate "debian_vbox" | {grep_utility} IP').stdout).group(1)

class TestCaseFindIPAddress1(TestCase):
    name = 'Viens fizisks interfeiss + loopback'
    level = 1

    def setup_functional(self, problem):
        debian_exec('ip link set docker0 down',
                    'ip link delete docker0')

    def run_functional_internal(self, problem, script_name):
        def find_issue(test, problem, results):
            if f'IP: {problem.correct_address}\n' not in results.stdout:
                if f'IP: {problem.correct_address}' in results.stdout:
                    return 'Only output the one address after "IP: " in one line', 'CONDITION_NOT_MET'
                if f'{problem.correct_address}\n' in results.stdout:
                    return 'The IP address was not formatted correctly', 'CONDITION_PARTIALLY_MET'
                return 'The correct IP address was not outputted', 'CONDITION_NOT_MET'
            
            return None, None
        
        test_exec(self, problem, script_name, find_issue, '/')

class TestCaseFindIPAddress2(TestCase):
    name = 'Viens fizisks interfeiss + loopback + virtuāls docker0 interfeiss'
    level = 2

    def setup_functional(self, problem):
        pass

    def run_functional_internal(self, problem, script_name):
        def find_issue(test, problem, results):
            if f'IP: {problem.correct_address}\n' not in results.stdout:
                if f'IP: {problem.correct_address}' in results.stdout:
                    return 'Only output the one address after "IP: " in one line. If there exist any virtual interfaces, only output the physical interface\'s IP', 'CONDITION_NOT_MET'
                if f'{problem.correct_address}\n' in results.stdout:
                    return 'The IP address was not formatted correctly', 'CONDITION_PARTIALLY_MET'
                return 'The correct IP address was not outputted. If there exist any virtual interfaces, only output the physical interface\'s IP', 'CONDITION_NOT_MET'
            
            return None, None
        
        test_exec(self, problem, script_name, find_issue, '/')

FindIPAddressInstance = FindIPAddress([
                                TestCaseFindIPAddress1(),
                                TestCaseFindIPAddress2()
                            ])
problems.append(FindIPAddressInstance)

<>:3705: SyntaxWarning: invalid escape sequence '\.'
<>:3705: SyntaxWarning: invalid escape sequence '\.'
C:\Users\Banknote\AppData\Local\Temp\ipykernel_185348\2803998255.py:3705: SyntaxWarning: invalid escape sequence '\.'
  self.correct_address = re.search("'([0-9\.]+)'", run_command(f'VBoxManage guestproperty enumerate "debian_vbox" | {grep_utility} IP').stdout).group(1)


# Start experiment

In [9]:
starting_problem_found = START_FROM_PROBLEM_NAME == None
last_finished_problem = None

def attempt_score(results):
    functional_score = 6
    has_failed_test = False
    for _, test in results['test_cases'].items():
        functional_score -= 6 * ISSUE_TYPE_WEIGHTS[test['functional_issue_type']] / len(results['test_cases'].items()) if not test['functional_test_success'] else 0
        has_failed_test = has_failed_test or not test['functional_test_success']
    if not has_failed_test:
        functional_score += 2

    if functional_score < 0:
        functional_score = 0

    static_score = 5
    static_score -= results['sh_style_cnt'] * 0.5
    static_score -= results['sh_info_cnt'] * 1
    static_score -= results['sh_warning_cnt'] * 2
    static_score -= results['sh_error_cnt'] * 3

    if static_score < 0:
        static_score = 0

    return functional_score + static_score

try:
    number_of_problems_tested = 0

    SCRIPTS_ALREADY_GENERATED = get_script_data_from_json()

    total_in_batch = MAX_NUMBER_OF_PROBLEMS_TO_TEST_IN_BATCH if len(problems) > MAX_NUMBER_OF_PROBLEMS_TO_TEST_IN_BATCH else len(problems)

    for i, problem in enumerate(problems):
        tested_cache = dict()
        if number_of_problems_tested >= MAX_NUMBER_OF_PROBLEMS_TO_TEST_IN_BATCH:
            print(f'Maximum number of problems to test in batch reached ({MAX_NUMBER_OF_PROBLEMS_TO_TEST_IN_BATCH}), stopping after finishing problem {last_finished_problem}')
            break
        if not starting_problem_found and (problem.name == START_FROM_PROBLEM_NAME or START_FROM_PROBLEM_NAME == None):
            total_in_batch = MAX_NUMBER_OF_PROBLEMS_TO_TEST_IN_BATCH if len(problems) - i > MAX_NUMBER_OF_PROBLEMS_TO_TEST_IN_BATCH else len(problems) - i
            starting_problem_found = True

        if starting_problem_found:
            if not SKIP_CORRECT_SCRIPTS:
                results_correct = problem.run_test_cases(f'correct_{problem.name}', skip_static=True)
                all_correct = True
                for _, test in results_correct['test_cases'].items():
                    all_correct = all_correct and test['functional_test_success']
                if not all_correct:
                    print(results_correct)
                    raise Exception(f'Correct example script for problem {problem.name} failed to execute correctly. There might be an issue with the tests.')

            if REDO_PROBLEMS:
                clearProblem(EXPERIMENT_NAME, problem.name)
            problem.generateScripts()
            shellcheck_copy_scripts() # Copy the generated scripts onto the shellcheck container for static analysis
            
            for script in problem.scripts:
                if not REDO_PROBLEMS:
                    if resultExists(EXPERIMENT_NAME, problem.name, script['model'], script['top-p'], script['temperature'], 1):
                        continue 

                regenerated_script = None
                for attempt in range(1, ATTEMPTS + 1):
                    if regenerated_script != None:
                        curr_script = regenerated_script
                    else:
                        curr_script = script

                    if curr_script['script'] == NOT_A_BASH:
                        for _ in range(attempt, ATTEMPTS + 1):
                            for test in problem.test_cases:
                                saveResultRow(EXPERIMENT_NAME, problem.name, problem.text, problem.category,
                                            problem.level, attempt, curr_script['script'], curr_script['temperature'], 
                                            curr_script['top-p'], curr_script['model'], test.name, test.level,
                                            error, success, 'N/A', 0, 
                                            0, 0, 0,
                                            'N/A', 0, curr_script['tokens_used'],
                                            curr_script['time_taken_ms'], 'NO_SCRIPT', problem.type)
                        break

                    # Often scripts are identical (especially for the same model with different parameters), so we can skip testing them again
                    if curr_script['script'] in tested_cache:
                        results = tested_cache[curr_script['script']]
                    else:
                        attempts_run_tests = 2
                        results = None
                        while attempts_run_tests > 0 and results == None:
                            try:
                                results = problem.run_test_cases(curr_script['name'])
                            except Exception as e:
                                attempts_run_tests -= 1
                                results = None
                                if attempts_run_tests == 0:
                                    raise e
                        tested_cache[curr_script['script']] = results
                    
                    static_issues = results['static_issues'] if results['static_issues'] else ''
                    success = results['static_clean']

                    first_functional_error = ""
                    first_functional_issue = ""
                    output_on_error = ""

                    for _, test in results['test_cases'].items():
                        error = test['functional_error'] if 'functional_error' in test else ''
                        functional_issue = test['functional_issue'] if 'functional_issue' in test else ''
                        success = success and test['functional_test_success']

                        if len(first_functional_issue) == 0 and len(functional_issue) > 0:
                            first_functional_issue = functional_issue
                            if len(error) > 0:
                                first_functional_error = error
                            output_on_error = test['output'] if 'output' in test else ''

                        saveResultRow(EXPERIMENT_NAME, problem.name, problem.text, problem.category,
                                    problem.level, attempt, curr_script['script'], curr_script['temperature'], 
                                    curr_script['top-p'], curr_script['model'], test['test_name'], test['test_level'],
                                    error, success, functional_issue, results['sh_style_cnt'], 
                                    results['sh_info_cnt'], results['sh_warning_cnt'], results['sh_error_cnt'],
                                    static_issues, attempt_score(results), curr_script['tokens_used'],
                                    curr_script['time_taken_ms'], test['functional_issue_type'] if 'functional_issue_type' in test else '', problem.type)
                        
                    if not success and attempt < ATTEMPTS:
                        issues_explanation = ''
                        if len(first_functional_issue) > 0:
                            issues_explanation += first_functional_issue + '\n'
 
                        if len(first_functional_error) > 0 and len(first_functional_issue) > 0:
                            issues_explanation += f'The stderr was { ' (truncated to last 500 characters) ' if len(first_functional_error) > 500 else ''}:\n {first_functional_error[-500:]}\n'
                        if len(first_functional_issue) > 0:
                            issues_explanation += f'The stdout was { ' (truncated to last 500 characters) ' if len(output_on_error) > 500 else ''}:\n {output_on_error[-500:]}\n'

                        if len(static_issues) > 0:
                            issues_explanation += f'A static analysis of the script found the following issues:\n{static_issues}\n'

                        regenerated_script = problem.regenerateScript(curr_script['model'], curr_script['temperature'], curr_script['top-p'], issues_explanation)
                        shellcheck_copy_scripts() # Recopy scripts to include the new script
                    else:
                        break
            last_finished_problem = problem.name
            number_of_problems_tested += 1
            print(f'[{number_of_problems_tested}/{total_in_batch}] Problem {problem.name} has been tested!')
            if i == len(problems) - 1:
                print('All problems have been tested! Results may be reviewed in the Postgres database or Apache Superset.')
                break
except Exception as e:
    print(f'Script terminated while working on problem {problem.name}, last successful problem was {last_finished_problem}')
    raise e

KeyboardInterrupt: 